In [ ]:
import sys
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/poc")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/plugins")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/dags")

#Import libs python
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import date

#Import libs internas
from utils import spark_utils_session as utils

from hooks.hdfs.hdfs_helper import HdfsHelper
from jobs.job_base_config import BaseETLJobClass

import poc_helper
poc_helper.load_env("PROD")

In [ ]:
def get_session(profile: str, dynamic_allocation_enabled: bool = True) -> utils.DBASparkAppSession:
    """Generates DBASparkAppSession."""
    
    app_name = "tsevero_ecd"
    
    spark_builder = (utils.DBASparkAppSession
                     .builder
                     .setAppName(app_name)
                     .usingProcessProfile(profile)
                    )
    
    if dynamic_allocation_enabled:
        spark_builder.autoResourceManagement()

    return spark_builder.build()

session = get_session(profile='efd_t2')

In [ ]:
session.sparkSession.sql("SHOW DATABASES").show(truncate=False)

In [ ]:
# ============================================================================
# CÉLULA 1: CONFIGURAÇÃO INICIAL - PROJETO ECD
# ============================================================================
# Escrituração Contábil Digital - Análise Completa
# Auditor Fiscal da Receita Estadual de SC
# ============================================================================

import sys
import warnings
from datetime import datetime, date
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# PySpark imports com aliases para evitar conflitos
from pyspark.sql.functions import (
    col as spark_col, 
    sum as spark_sum, 
    avg as spark_avg,
    count as spark_count,
    when as spark_when,
    desc as spark_desc,
    asc as spark_asc,
    round as spark_round,
    coalesce as spark_coalesce,
    max as spark_max,
    min as spark_min,
    stddev as spark_stddev,
    lit as spark_lit,
    concat as spark_concat,
    expr as spark_expr,
    abs as spark_abs
)

# Machine Learning imports
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    roc_auc_score, 
    silhouette_score,
    davies_bouldin_score,
    calinski_harabasz_score
)
import xgboost as xgb

# Acesso ao SparkSession
spark = session.sparkSession

# Configurações
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

# Configuração de estilo para gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Verificar se spark existe na sessão
try:
    spark
    print("✅ Sessão Spark já ativa!")
except NameError:
    print("⚠️ Sessão Spark não encontrada. Por favor, inicialize a sessão Spark.")

print("\n" + "="*80)
print("🚀 PROJETO ECD - ANÁLISE EXPLORATÓRIA E MACHINE LEARNING")
print("="*80)
print(f"📅 Data de execução: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
print("📊 Bibliotecas carregadas com sucesso!")
print("="*80 + "\n")

In [ ]:
# ============================================================================
# CÉLULA 2: VISÃO GERAL DOS DADOS ECD (Versão com TQDM e Status)
# ============================================================================

# --- Importações Necessárias ---
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm.notebook import tqdm

# Assumindo que a variável 'spark' já está definida no seu ambiente.

print("\n" + "="*80)
print("📊 VISÃO GERAL - TABELAS ECD")
print("="*80)

# ============================================================================
# CONTAGEM DE REGISTROS POR TABELA
# ============================================================================

tabelas_ecd = [
    'teste.ecd_empresas_cadastro',
    'teste.ecd_plano_contas',
    'teste.ecd_saldos_contas',
    'teste.ecd_balanco_patrimonial',
    'teste.ecd_dre',
    'teste.ecd_indicadores_financeiros',
    'teste.ecd_inconsistencias_equacao',
    'teste.ecd_benchmark_setorial',  # Esta não tem CNPJ!
    'teste.ecd_score_risco_consolidado'
]

# Tabelas que NÃO têm coluna CNPJ (são agregadas)
tabelas_sem_cnpj = ['teste.ecd_benchmark_setorial']

resultados = []

print("Iniciando varredura das tabelas (isso pode demorar muito)...")

# Envolvemos a lista de tabelas com o tqdm para uma barra de progresso
for tabela in tqdm(tabelas_ecd, desc="Progresso Geral (Tabelas)"):
    tabela_nome = tabela.split('.')[-1]
    
    try:
        print(f"\n[Processando] Tabela '{tabela_nome}'...")
        
        # --- Consulta 1: COUNT(*) ---
        print(f"  (1/2) Executando COUNT(*)...")
        total = spark.sql(f"SELECT COUNT(*) as cnt FROM {tabela}").collect()[0]['cnt']
        
        # --- Consulta 2: COUNT(DISTINCT) - com tratamento especial ---
        if tabela in tabelas_sem_cnpj:
            print(f"  (2/2) Tabela agregada (sem CNPJ) - pulando COUNT(DISTINCT cnpj)...")
            empresas = "N/A (agregada)"
        else:
            print(f"  (2/2) Executando COUNT(DISTINCT cnpj)... (Esta é a parte MAIS LENTA)")
            empresas = spark.sql(f"SELECT COUNT(DISTINCT cnpj) as cnt FROM {tabela}").collect()[0]['cnt']
            empresas = f'{empresas:,}'
        
        resultados.append({
            'Tabela': tabela_nome,
            'Total Registros': f'{total:,}',
            'Empresas Únicas': empresas
        })
        
        print(f"✅ [Concluído] {tabela_nome}: {total:,} registros | {empresas} empresas")
        
    except Exception as e:
        print(f"⚠️ Erro ao acessar {tabela}: {str(e)}")
        resultados.append({
            'Tabela': tabela_nome,
            'Total Registros': 'Erro',
            'Empresas Únicas': 'Erro'
        })

# Criar DataFrame resumo
df_resumo = pd.DataFrame(resultados)

print("\n" + "="*80)
print("📈 RESUMO DAS TABELAS")
print("="*80)
print(df_resumo.to_string(index=False))

# ============================================================================
# DISTRIBUIÇÃO POR ANO REFERÊNCIA
# ============================================================================

print("\n" + "="*80)
print("📅 DISTRIBUIÇÃO POR ANO REFERÊNCIA")
print("="*80)

print("Iniciando consulta de distribuição por ano (GROUP BY com COUNT DISTINCT)...")
# Esta consulta também é pesada e mostrará a barra [Stage 0:>]
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW vw_anos_ref AS
SELECT 
    CAST(ano_referencia / 100 AS INT) AS ano_fiscal,
    COUNT(DISTINCT cnpj) AS qtd_empresas,
    COUNT(DISTINCT id_ecd) AS qtd_ecds
FROM teste.ecd_empresas_cadastro
WHERE ano_referencia > 0
GROUP BY CAST(ano_referencia / 100 AS INT)
ORDER BY ano_fiscal DESC
""")
print("...Consulta de distribuição por ano CONCLUÍDA.")

# Esta é uma consulta leve (na view temporária)
total_anos = spark.sql("SELECT COUNT(*) as cnt FROM vw_anos_ref").collect()[0]['cnt']

if total_anos > 0 and total_anos <= 20:
    print("Buscando dados da view para o gráfico (executando .toPandas())...")
    # O .toPandas() é uma AÇÃO e vai disparar o job do Spark
    df_anos = spark.sql("SELECT * FROM vw_anos_ref").toPandas()
    print("...Dados recebidos. Gerando visualização.")
    
    print("\n📊 Anos disponíveis na base:")
    for _, row in df_anos.iterrows():
        print(f"  • {row['ano_fiscal']}: {row['qtd_empresas']:,} empresas | {row['qtd_ecds']:,} ECDs")
    
    # Gráfico de evolução
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Empresas por Ano', 'ECDs por Ano'),
        specs=[[{'type': 'bar'}, {'type': 'bar'}]]
    )
    
    fig.add_trace(
        go.Bar(
            x=df_anos['ano_fiscal'],
            y=df_anos['qtd_empresas'],
            name='Empresas',
            marker=dict(color='#1f77b4'),
            text=df_anos['qtd_empresas'],
            textposition='outside'
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Bar(
            x=df_anos['ano_fiscal'],
            y=df_anos['qtd_ecds'],
            name='ECDs',
            marker=dict(color='#ff7f0e'),
            text=df_anos['qtd_ecds'],
            textposition='outside'
        ),
        row=1, col=2
    )
    
    fig.update_layout(
        title='<b>Distribuição Temporal - Base ECD</b>',
        height=400,
        showlegend=False
    )
    
    fig.update_xaxes(title_text="Ano Fiscal", row=1, col=1)
    fig.update_xaxes(title_text="Ano Fiscal", row=1, col=2)
    fig.update_yaxes(title_text="Quantidade", row=1, col=1)
    fig.update_yaxes(title_text="Quantidade", row=1, col=2)
    
    fig.show()

else:
    print(f"⚠️ Distribuição temporal não disponível ({total_anos} registros)")

print("\n✅ Visão geral concluída!")

In [ ]:
# ============================================================================
# CÉLULA 3: ANÁLISE BALANÇO PATRIMONIAL E DRE
# ============================================================================

print("\n" + "="*80)
print("💰 ANÁLISE FINANCEIRA - BALANÇO PATRIMONIAL E DRE")
print("="*80)

# ============================================================================
# PREPARAR VIEW DE ANÁLISE FINANCEIRA
# ============================================================================

spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW vw_analise_financeira AS
SELECT 
    bp.cnpj,
    ec.nm_razao_social,
    ec.cd_uf,
    ec.cnae_secao,
    ec.cnae_secao_descricao,
    CAST(bp.ano_referencia / 100 AS INT) AS ano_fiscal,
    CAST(COALESCE(bp.ativo_total, 0) AS DOUBLE) AS ativo_total,
    CAST(COALESCE(bp.ativo_circulante, 0) AS DOUBLE) AS ativo_circulante,
    CAST(COALESCE(bp.ativo_nao_circulante, 0) AS DOUBLE) AS ativo_nao_circulante,
    CAST(COALESCE(bp.passivo_circulante, 0) AS DOUBLE) AS passivo_circulante,
    CAST(COALESCE(bp.passivo_nao_circulante, 0) AS DOUBLE) AS passivo_nao_circulante,
    CAST(COALESCE(bp.patrimonio_liquido, 0) AS DOUBLE) AS patrimonio_liquido,
    CAST(COALESCE(dre.receita_bruta, 0) AS DOUBLE) AS receita_bruta,
    CAST(COALESCE(dre.receita_liquida, 0) AS DOUBLE) AS receita_liquida,
    CAST(COALESCE(dre.custos_totais, 0) AS DOUBLE) AS custos_totais,
    CAST(COALESCE(dre.despesas_totais, 0) AS DOUBLE) AS despesas_totais,
    CAST(COALESCE(dre.lucro_bruto, 0) AS DOUBLE) AS lucro_bruto,
    CAST(COALESCE(dre.resultado_liquido, 0) AS DOUBLE) AS resultado_liquido
FROM teste.ecd_balanco_patrimonial bp
INNER JOIN teste.ecd_empresas_cadastro ec 
    ON bp.cnpj = ec.cnpj 
    AND bp.ano_referencia = ec.ano_referencia
LEFT JOIN teste.ecd_dre dre 
    ON bp.cnpj = dre.cnpj 
    AND bp.ano_referencia = dre.ano_referencia
WHERE bp.ativo_total > 0
""")

# Verificar tamanho
total_fin = spark.sql("SELECT COUNT(*) as cnt FROM vw_analise_financeira").collect()[0]['cnt']
print(f"\n📊 Total de registros financeiros: {total_fin:,}")

# ============================================================================
# ESTATÍSTICAS DESCRITIVAS PRINCIPAIS
# ============================================================================

if total_fin > 0:
    # Limitar a 50.000 registros se necessário
    if total_fin > 50000:
        print(f"⚠️ Muitos registros ({total_fin:,}), limitando a 50.000 para análise...")
        df_financeiro = spark.sql("SELECT * FROM vw_analise_financeira ORDER BY ativo_total DESC LIMIT 50000").toPandas()
    else:
        df_spark = spark.sql("SELECT * FROM vw_analise_financeira")
        df_spark.cache()
        df_financeiro = df_spark.toPandas()
    
    print(f"\n✅ Dados carregados: {len(df_financeiro):,} registros")
    
    # ========================================================================
    # ESTATÍSTICAS DESCRITIVAS
    # ========================================================================
    
    print("\n" + "="*80)
    print("📈 ESTATÍSTICAS DESCRITIVAS - VALORES EM MILHÕES (R$)")
    print("="*80)
    
    colunas_valores = [
        'ativo_total', 'ativo_circulante', 'ativo_nao_circulante',
        'passivo_circulante', 'passivo_nao_circulante', 'patrimonio_liquido',
        'receita_bruta', 'receita_liquida', 'lucro_bruto', 'resultado_liquido'
    ]
    
    # Converter para milhões
    df_stats = df_financeiro[colunas_valores].copy()
    df_stats = df_stats / 1_000_000  # Converter para milhões
    
    # Calcular estatísticas
    stats = df_stats.describe()
    
    print("\n📊 Resumo Estatístico (em Milhões R$):")
    print(stats.to_string())
    
    # ========================================================================
    # GRÁFICO: DISTRIBUIÇÃO DE ATIVOS (TOP 30 EMPRESAS)
    # ========================================================================
    
    print("\n" + "="*80)
    print("📊 VISUALIZAÇÃO: TOP 30 EMPRESAS POR ATIVO TOTAL")
    print("="*80)
    
    df_top30 = df_financeiro.nlargest(30, 'ativo_total').copy()
    df_top30['ativo_milhoes'] = df_top30['ativo_total'] / 1_000_000
    df_top30['razao_curta'] = df_top30['nm_razao_social'].str[:40]
    
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        y=df_top30['razao_curta'],
        x=df_top30['ativo_milhoes'],
        orientation='h',
        marker=dict(
            color=df_top30['ativo_milhoes'],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title="Ativo (Mi R$)")
        ),
        text=[f"R$ {val:,.1f}M" for val in df_top30['ativo_milhoes']],
        textposition='outside',
        hovertemplate='<b>%{y}</b><br>Ativo: R$ %{x:,.2f}M<extra></extra>'
    ))
    
    fig.update_layout(
        title='<b>Top 30 Empresas - Ativo Total</b>',
        xaxis_title='Ativo Total (Milhões R$)',
        yaxis_title='Empresa',
        height=800,
        showlegend=False,
        yaxis=dict(autorange='reversed')
    )
    
    fig.show()
    
    # ========================================================================
    # GRÁFICO: COMPOSIÇÃO MÉDIA DO BALANÇO
    # ========================================================================
    
    print("\n" + "="*80)
    print("📊 COMPOSIÇÃO MÉDIA DO BALANÇO PATRIMONIAL")
    print("="*80)
    
    composicao_ativo = {
        'Ativo Circulante': df_financeiro['ativo_circulante'].sum(),
        'Ativo Não Circulante': df_financeiro['ativo_nao_circulante'].sum()
    }
    
    composicao_passivo = {
        'Passivo Circulante': df_financeiro['passivo_circulante'].sum(),
        'Passivo Não Circulante': df_financeiro['passivo_nao_circulante'].sum(),
        'Patrimônio Líquido': df_financeiro['patrimonio_liquido'].sum()
    }
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Composição do ATIVO', 'Composição do PASSIVO + PL'),
        specs=[[{'type': 'pie'}, {'type': 'pie'}]]
    )
    
    fig.add_trace(
        go.Pie(
            labels=list(composicao_ativo.keys()),
            values=list(composicao_ativo.values()),
            hole=0.4,
            marker=dict(colors=['#1f77b4', '#ff7f0e'])
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Pie(
            labels=list(composicao_passivo.keys()),
            values=list(composicao_passivo.values()),
            hole=0.4,
            marker=dict(colors=['#2ca02c', '#d62728', '#9467bd'])
        ),
        row=1, col=2
    )
    
    fig.update_layout(
        title='<b>Composição Patrimonial - Base Agregada</b>',
        height=500,
        showlegend=True
    )
    
    fig.show()
    
    # ========================================================================
    # ANÁLISE DRE - RENTABILIDADE
    # ========================================================================
    
    print("\n" + "="*80)
    print("💹 ANÁLISE DE RENTABILIDADE (DRE)")
    print("="*80)
    
    # Filtrar empresas com receita > 0
    df_dre = df_financeiro[df_financeiro['receita_liquida'] > 0].copy()
    
    if len(df_dre) > 0:
        # Calcular margens
        df_dre['margem_bruta'] = (df_dre['lucro_bruto'] / df_dre['receita_liquida']) * 100
        df_dre['margem_liquida'] = (df_dre['resultado_liquido'] / df_dre['receita_liquida']) * 100
        
        print(f"\n✅ {len(df_dre):,} empresas com receita informada")
        print(f"\n📊 Margem Bruta Média: {df_dre['margem_bruta'].mean():.2f}%")
        print(f"📊 Margem Líquida Média: {df_dre['margem_liquida'].mean():.2f}%")
        print(f"📊 Empresas com lucro: {(df_dre['resultado_liquido'] > 0).sum():,} ({(df_dre['resultado_liquido'] > 0).sum() / len(df_dre) * 100:.1f}%)")
        print(f"📊 Empresas com prejuízo: {(df_dre['resultado_liquido'] < 0).sum():,} ({(df_dre['resultado_liquido'] < 0).sum() / len(df_dre) * 100:.1f}%)")
        
        # Gráfico de distribuição de margens
        fig = make_subplots(
            rows=1, cols=2,
            subplot_titles=('Distribuição - Margem Bruta', 'Distribuição - Margem Líquida')
        )
        
        fig.add_trace(
            go.Histogram(
                x=df_dre['margem_bruta'].clip(-100, 100),
                nbinsx=50,
                name='Margem Bruta',
                marker=dict(color='#1f77b4')
            ),
            row=1, col=1
        )
        
        fig.add_trace(
            go.Histogram(
                x=df_dre['margem_liquida'].clip(-100, 100),
                nbinsx=50,
                name='Margem Líquida',
                marker=dict(color='#ff7f0e')
            ),
            row=1, col=2
        )
        
        fig.update_xaxes(title_text="Margem (%)", row=1, col=1)
        fig.update_xaxes(title_text="Margem (%)", row=1, col=2)
        fig.update_yaxes(title_text="Frequência", row=1, col=1)
        fig.update_yaxes(title_text="Frequência", row=1, col=2)
        
        fig.update_layout(
            title='<b>Distribuição de Margens - Análise de Rentabilidade</b>',
            height=400,
            showlegend=False
        )
        
        fig.show()
    
    else:
        print("\n⚠️ Nenhuma empresa com receita informada para análise DRE")

else:
    print("\n⚠️ Dados financeiros não disponíveis")

print("\n✅ Análise financeira concluída!")

In [ ]:
# ============================================================================
# CÉLULA 4: ANÁLISE DE INDICADORES FINANCEIROS
# ============================================================================

print("\n" + "="*80)
print("📊 ANÁLISE DE INDICADORES FINANCEIROS")
print("="*80)

# ============================================================================
# PREPARAR VIEW DE INDICADORES
# ============================================================================

spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW vw_indicadores AS
SELECT 
    ind.cnpj,
    ec.nm_razao_social,
    ec.cd_uf,
    ec.cnae_secao,
    ec.cnae_secao_descricao,
    ec.empresa_grande_porte,
    CAST(ind.ano_referencia / 100 AS INT) AS ano_fiscal,
    CAST(COALESCE(ind.ativo_total, 0) AS DOUBLE) AS ativo_total,
    CAST(COALESCE(ind.receita_liquida, 0) AS DOUBLE) AS receita_liquida,
    CAST(COALESCE(ind.resultado_liquido, 0) AS DOUBLE) AS resultado_liquido,
    CAST(COALESCE(ind.liquidez_corrente, 0) AS DOUBLE) AS liquidez_corrente,
    CAST(COALESCE(ind.liquidez_geral, 0) AS DOUBLE) AS liquidez_geral,
    CAST(COALESCE(ind.endividamento_geral, 0) AS DOUBLE) AS endividamento_geral,
    CAST(COALESCE(ind.composicao_endividamento, 0) AS DOUBLE) AS composicao_endividamento,
    CAST(COALESCE(ind.margem_liquida_perc, 0) AS DOUBLE) AS margem_liquida_perc,
    CAST(COALESCE(ind.margem_bruta_perc, 0) AS DOUBLE) AS margem_bruta_perc,
    CAST(COALESCE(ind.roa_retorno_ativo_perc, 0) AS DOUBLE) AS roa_retorno_ativo_perc,
    CAST(COALESCE(ind.roe_retorno_patrimonio_perc, 0) AS DOUBLE) AS roe_retorno_patrimonio_perc
FROM teste.ecd_indicadores_financeiros ind
INNER JOIN teste.ecd_empresas_cadastro ec 
    ON ind.cnpj = ec.cnpj 
    AND ind.ano_referencia = ec.ano_referencia
WHERE ind.ativo_total > 0
    AND ind.liquidez_corrente IS NOT NULL
""")

# Verificar tamanho
total_ind = spark.sql("SELECT COUNT(*) as cnt FROM vw_indicadores").collect()[0]['cnt']
print(f"\n📊 Total de registros com indicadores: {total_ind:,}")

if total_ind > 0:
    # Limitar a 30.000 registros se necessário
    if total_ind > 30000:
        print(f"⚠️ Muitos registros ({total_ind:,}), limitando a 30.000...")
        df_ind = spark.sql("SELECT * FROM vw_indicadores ORDER BY ativo_total DESC LIMIT 30000").toPandas()
    else:
        df_spark = spark.sql("SELECT * FROM vw_indicadores")
        df_spark.cache()
        df_ind = df_spark.toPandas()
    
    print(f"✅ Dados carregados: {len(df_ind):,} registros")
    
    # ========================================================================
    # ESTATÍSTICAS DOS INDICADORES
    # ========================================================================
    
    print("\n" + "="*80)
    print("📈 ESTATÍSTICAS DOS INDICADORES FINANCEIROS")
    print("="*80)
    
    indicadores_cols = [
        'liquidez_corrente', 'liquidez_geral', 'endividamento_geral',
        'margem_liquida_perc', 'margem_bruta_perc', 
        'roa_retorno_ativo_perc', 'roe_retorno_patrimonio_perc'
    ]
    
    stats_ind = df_ind[indicadores_cols].describe()
    print("\n" + stats_ind.to_string())
    
    # ========================================================================
    # CLASSIFICAÇÃO DE LIQUIDEZ
    # ========================================================================
    
    df_ind['classe_liquidez'] = pd.cut(
        df_ind['liquidez_corrente'],
        bins=[0, 0.5, 1.0, 1.5, 2.0, float('inf')],
        labels=['Crítica', 'Baixa', 'Regular', 'Boa', 'Excelente']
    )
    
    df_ind['classe_endividamento'] = pd.cut(
        df_ind['endividamento_geral'],
        bins=[0, 0.3, 0.5, 0.7, 1.0, float('inf')],
        labels=['Baixo', 'Moderado', 'Alto', 'Crítico', 'Muito Crítico']
    )
    
    print("\n" + "="*80)
    print("🎯 CLASSIFICAÇÃO DE LIQUIDEZ")
    print("="*80)
    
    dist_liquidez = df_ind['classe_liquidez'].value_counts().sort_index()
    print("\n" + dist_liquidez.to_string())
    
    print("\n" + "="*80)
    print("⚠️ CLASSIFICAÇÃO DE ENDIVIDAMENTO")
    print("="*80)
    
    dist_endiv = df_ind['classe_endividamento'].value_counts().sort_index()
    print("\n" + dist_endiv.to_string())
    
    # ========================================================================
    # GRÁFICO: MATRIZ DE CORRELAÇÃO DE INDICADORES
    # ========================================================================
    
    print("\n" + "="*80)
    print("📊 VISUALIZAÇÃO: MATRIZ DE CORRELAÇÃO DE INDICADORES")
    print("="*80)
    
    # Selecionar amostra para visualização se dataset muito grande
    # FIX: Usar min() do Python nativo, não do PySpark
    tamanho_amostra = 1000 if len(df_ind) > 1000 else len(df_ind)
    df_sample = df_ind.sample(n=tamanho_amostra, random_state=42)
    
    # Limpar outliers extremos para melhor visualização
    for col in indicadores_cols:
        q1 = df_sample[col].quantile(0.05)
        q3 = df_sample[col].quantile(0.95)
        df_sample.loc[df_sample[col] < q1, col] = q1
        df_sample.loc[df_sample[col] > q3, col] = q3
    
    # Calcular correlação
    corr_matrix = df_sample[indicadores_cols].corr()
    
    # Heatmap de correlação
    fig = go.Figure(data=go.Heatmap(
        z=corr_matrix.values,
        x=['Liq.Cor.', 'Liq.Geral', 'Endiv.', 'M.Líq.%', 'M.Bruta%', 'ROA%', 'ROE%'],
        y=['Liq.Cor.', 'Liq.Geral', 'Endiv.', 'M.Líq.%', 'M.Bruta%', 'ROA%', 'ROE%'],
        colorscale='RdBu',
        zmid=0,
        text=corr_matrix.values.round(2),
        texttemplate='%{text}',
        textfont={"size": 10},
        colorbar=dict(title="Correlação")
    ))
    
    fig.update_layout(
        title='<b>Matriz de Correlação - Indicadores Financeiros</b>',
        height=600,
        xaxis_title='',
        yaxis_title=''
    )
    
    fig.show()
    
    # ========================================================================
    # GRÁFICO: DISTRIBUIÇÃO DE LIQUIDEZ x ENDIVIDAMENTO
    # ========================================================================
    
    print("\n" + "="*80)
    print("📊 ANÁLISE: LIQUIDEZ vs ENDIVIDAMENTO")
    print("="*80)
    
    # Preparar dados para scatter
    df_scatter = df_ind[
        (df_ind['liquidez_corrente'] > 0) & 
        (df_ind['liquidez_corrente'] < 10) &
        (df_ind['endividamento_geral'] >= 0) &
        (df_ind['endividamento_geral'] <= 2)
    ].copy()
    
    df_scatter['tamanho'] = df_scatter['ativo_total'] / 1_000_000  # Em milhões
    
    # FIX: Usar min() do Python
    tamanho_scatter = 2000 if len(df_scatter) > 2000 else len(df_scatter)
    df_scatter_sample = df_scatter.sample(n=tamanho_scatter, random_state=42)
    
    fig = px.scatter(
        df_scatter_sample,
        x='endividamento_geral',
        y='liquidez_corrente',
        color='margem_liquida_perc',
        size='tamanho',
        hover_data=['nm_razao_social', 'cd_uf', 'cnae_secao_descricao'],
        color_continuous_scale='RdYlGn',
        labels={
            'endividamento_geral': 'Endividamento Geral',
            'liquidez_corrente': 'Liquidez Corrente',
            'margem_liquida_perc': 'Margem Líquida (%)'
        },
        title='<b>Liquidez vs Endividamento (tamanho = Ativo Total)</b>'
    )
    
    # Adicionar linhas de referência
    fig.add_hline(y=1.0, line_dash="dash", line_color="red", 
                  annotation_text="Liquidez = 1.0", annotation_position="right")
    fig.add_vline(x=0.7, line_dash="dash", line_color="orange",
                  annotation_text="Endiv. = 0.7", annotation_position="top")
    
    fig.update_layout(height=600)
    fig.show()
    
    # ========================================================================
    # GRÁFICO: BOX PLOT POR SETOR
    # ========================================================================
    
    print("\n" + "="*80)
    print("📊 INDICADORES POR SETOR (TOP 10 SETORES)")
    print("="*80)
    
    # Top 10 setores
    top_setores = df_ind['cnae_secao_descricao'].value_counts().head(10).index.tolist()
    df_setores = df_ind[df_ind['cnae_secao_descricao'].isin(top_setores)].copy()
    
    # Preparar nomes curtos
    df_setores['setor_curto'] = df_setores['cnae_secao_descricao'].str[:40]
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Liquidez Corrente por Setor',
            'Endividamento por Setor',
            'Margem Líquida por Setor',
            'ROE por Setor'
        )
    )
    
    for setor in df_setores['setor_curto'].unique():
        dados_setor = df_setores[df_setores['setor_curto'] == setor]
        
        fig.add_trace(
            go.Box(y=dados_setor['liquidez_corrente'].clip(0, 5), name=setor, showlegend=False),
            row=1, col=1
        )
        fig.add_trace(
            go.Box(y=dados_setor['endividamento_geral'].clip(0, 1.5), name=setor, showlegend=False),
            row=1, col=2
        )
        fig.add_trace(
            go.Box(y=dados_setor['margem_liquida_perc'].clip(-50, 50), name=setor, showlegend=False),
            row=2, col=1
        )
        fig.add_trace(
            go.Box(y=dados_setor['roe_retorno_patrimonio_perc'].clip(-50, 50), name=setor, showlegend=False),
            row=2, col=2
        )
    
    fig.update_layout(
        title='<b>Distribuição de Indicadores por Setor</b>',
        height=800,
        showlegend=False
    )
    
    fig.show()
    
    # ========================================================================
    # ANÁLISE POR PORTE DE EMPRESA
    # ========================================================================
    
    if 'empresa_grande_porte' in df_ind.columns:
        print("\n" + "="*80)
        print("🏢 COMPARAÇÃO: GRANDE PORTE vs DEMAIS")
        print("="*80)
        
        df_porte = df_ind.groupby('empresa_grande_porte')[indicadores_cols].mean()
        print("\n📊 Médias por Porte:")
        print(df_porte.round(2).to_string())

else:
    print("\n⚠️ Dados de indicadores não disponíveis")

print("\n✅ Análise de indicadores concluída!")

In [ ]:
# ============================================================================
# CÉLULA 5: ANÁLISE DE SCORE DE RISCO E NEAF
# ============================================================================

print("\n" + "="*80)
print("⚠️ ANÁLISE DE RISCO E INDÍCIOS NEAF")
print("="*80)

# ============================================================================
# PREPARAR VIEW DE SCORE DE RISCO
# ============================================================================

spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW vw_score_risco AS
SELECT 
    sr.cnpj,
    sr.razao_social,
    sr.uf,
    sr.cd_cnae,
    sr.de_cnae,
    sr.cnae_secao,
    sr.cnae_secao_descricao,
    CAST(sr.ano_referencia / 100 AS INT) AS ano_fiscal,
    sr.empresa_grande_porte,
    sr.tipo_ecd,
    sr.nm_tipo_contribuinte,
    sr.classificacao_risco,
    CAST(COALESCE(sr.score_risco_total, 0) AS DOUBLE) AS score_risco_total,
    CAST(COALESCE(sr.score_equacao_contabil, 0) AS DOUBLE) AS score_equacao,
    CAST(COALESCE(sr.score_neaf, 0) AS DOUBLE) AS score_neaf,
    CAST(COALESCE(sr.score_risco_financeiro, 0) AS DOUBLE) AS score_financeiro,
    CAST(COALESCE(sr.qtd_indicios_neaf, 0) AS DOUBLE) AS qtd_indicios,
    CAST(COALESCE(sr.liquidez_corrente, 0) AS DOUBLE) AS liquidez_corrente,
    CAST(COALESCE(sr.endividamento_geral, 0) AS DOUBLE) AS endividamento_geral,
    CAST(COALESCE(sr.margem_liquida_perc, 0) AS DOUBLE) AS margem_liquida_perc,
    sr.posicao_liquidez_setor,
    sr.posicao_margem_setor
FROM teste.ecd_score_risco_consolidado sr
WHERE sr.score_risco_total IS NOT NULL
""")

# Verificar tamanho
total_score = spark.sql("SELECT COUNT(*) as cnt FROM vw_score_risco").collect()[0]['cnt']
print(f"\n📊 Total de registros com score: {total_score:,}")

if total_score > 0:
    # Limitar a 50.000 registros
    if total_score > 50000:
        print(f"⚠️ Muitos registros ({total_score:,}), limitando a 50.000...")
        df_score = spark.sql("SELECT * FROM vw_score_risco ORDER BY score_risco_total DESC LIMIT 50000").toPandas()
    else:
        df_spark = spark.sql("SELECT * FROM vw_score_risco")
        df_spark.cache()
        df_score = df_spark.toPandas()
    
    print(f"✅ Dados carregados: {len(df_score):,} registros")
    
    # ========================================================================
    # DISTRIBUIÇÃO DE CLASSIFICAÇÃO DE RISCO
    # ========================================================================
    
    print("\n" + "="*80)
    print("🎯 DISTRIBUIÇÃO DE CLASSIFICAÇÃO DE RISCO")
    print("="*80)
    
    dist_risco = df_score['classificacao_risco'].value_counts()
    print("\n" + dist_risco.to_string())
    
    # Percentuais
    print("\n📊 Percentuais:")
    for classe, qtd in dist_risco.items():
        pct = (qtd / len(df_score)) * 100
        print(f"  • {classe}: {pct:.1f}%")
    
    # Gráfico Pizza
    fig = go.Figure(data=[go.Pie(
        labels=dist_risco.index,
        values=dist_risco.values,
        hole=0.4,
        marker=dict(colors=['#d62728', '#ff7f0e', '#ffbb33', '#2ca02c']),
        textinfo='label+percent',
        textfont_size=14
    )])
    
    fig.update_layout(
        title='<b>Distribuição de Classificação de Risco</b>',
        height=500
    )
    
    fig.show()
    
    # ========================================================================
    # ESTATÍSTICAS DOS SCORES
    # ========================================================================
    
    print("\n" + "="*80)
    print("📈 ESTATÍSTICAS DOS SCORES")
    print("="*80)
    
    scores_cols = ['score_risco_total', 'score_equacao', 'score_neaf', 'score_financeiro']
    stats_scores = df_score[scores_cols].describe()
    print("\n" + stats_scores.to_string())
    
    # ========================================================================
    # ANÁLISE DE INDÍCIOS NEAF
    # ========================================================================
    
    print("\n" + "="*80)
    print("🚨 ANÁLISE DE INDÍCIOS NEAF")
    print("="*80)
    
    empresas_com_indicios = (df_score['qtd_indicios'] > 0).sum()
    pct_indicios = (empresas_com_indicios / len(df_score)) * 100
    
    print(f"\n📊 Empresas com indícios NEAF: {empresas_com_indicios:,} ({pct_indicios:.1f}%)")
    print(f"📊 Total de indícios: {df_score['qtd_indicios'].sum():.0f}")
    print(f"📊 Média de indícios (empresas com indício): {df_score[df_score['qtd_indicios'] > 0]['qtd_indicios'].mean():.1f}")
    
    # Distribuição de quantidade de indícios
    if empresas_com_indicios > 0:
        df_indicios = df_score[df_score['qtd_indicios'] > 0].copy()
        dist_indicios = df_indicios['qtd_indicios'].value_counts().sort_index()
        
        print("\n📊 Distribuição de Quantidade de Indícios:")
        for qtd, freq in dist_indicios.head(10).items():
            print(f"  • {int(qtd)} indícios: {freq:,} empresas")
    
    # ========================================================================
    # GRÁFICO: COMPOSIÇÃO DO SCORE DE RISCO
    # ========================================================================
    
    print("\n" + "="*80)
    print("📊 VISUALIZAÇÃO: COMPOSIÇÃO DO SCORE DE RISCO")
    print("="*80)
    
    # Médias por componente
    media_scores = {
        'Score Equação': df_score['score_equacao'].mean(),
        'Score NEAF': df_score['score_neaf'].mean(),
        'Score Financeiro': df_score['score_financeiro'].mean()
    }
    
    fig = go.Figure(data=[
        go.Bar(
            x=list(media_scores.keys()),
            y=list(media_scores.values()),
            marker=dict(
                color=list(media_scores.values()),
                colorscale='Reds',
                showscale=False
            ),
            text=[f"{val:.2f}" for val in media_scores.values()],
            textposition='outside'
        )
    ])
    
    fig.update_layout(
        title='<b>Composição Média do Score de Risco</b>',
        xaxis_title='Componente',
        yaxis_title='Score Médio',
        height=400
    )
    
    fig.show()
    
    # ========================================================================
    # GRÁFICO: SCORE DE RISCO POR UF (TOP 15)
    # ========================================================================
    
    print("\n" + "="*80)
    print("🗺️ SCORE DE RISCO POR UF")
    print("="*80)
    
    df_uf = df_score.groupby('uf').agg({
        'score_risco_total': 'mean',
        'cnpj': 'count',
        'qtd_indicios': 'sum'
    }).reset_index()
    
    df_uf.columns = ['UF', 'Score Médio', 'Qtd Empresas', 'Total Indícios']
    df_uf = df_uf.sort_values('Score Médio', ascending=False).head(15)
    
    print("\n📊 Top 15 UF com maior score de risco:")
    print(df_uf.to_string(index=False))
    
    fig = go.Figure(data=[
        go.Bar(
            x=df_uf['UF'],
            y=df_uf['Score Médio'],
            marker=dict(
                color=df_uf['Score Médio'],
                colorscale='Reds',
                showscale=True,
                colorbar=dict(title="Score")
            ),
            text=df_uf['Score Médio'].round(2),
            textposition='outside',
            hovertemplate='<b>%{x}</b><br>Score: %{y:.2f}<br>Empresas: %{customdata[0]:,}<br>Indícios: %{customdata[1]:,.0f}<extra></extra>',
            customdata=df_uf[['Qtd Empresas', 'Total Indícios']].values
        )
    ])
    
    fig.update_layout(
        title='<b>Score de Risco Médio por UF (Top 15)</b>',
        xaxis_title='UF',
        yaxis_title='Score Médio',
        height=500
    )
    
    fig.show()
    
    # ========================================================================
    # GRÁFICO: SCORE POR SETOR (TOP 20)
    # ========================================================================
    
    print("\n" + "="*80)
    print("🏭 SCORE DE RISCO POR SETOR")
    print("="*80)
    
    df_setor = df_score.groupby('cnae_secao_descricao').agg({
        'score_risco_total': 'mean',
        'cnpj': 'count',
        'qtd_indicios': 'sum'
    }).reset_index()
    
    df_setor.columns = ['Setor', 'Score Médio', 'Qtd Empresas', 'Total Indícios']
    df_setor = df_setor.sort_values('Score Médio', ascending=False).head(20)
    df_setor['Setor Curto'] = df_setor['Setor'].str[:50]
    
    fig = go.Figure(data=[
        go.Bar(
            y=df_setor['Setor Curto'],
            x=df_setor['Score Médio'],
            orientation='h',
            marker=dict(
                color=df_setor['Score Médio'],
                colorscale='Reds',
                showscale=False
            ),
            text=df_setor['Score Médio'].round(2),
            textposition='outside',
            hovertemplate='<b>%{y}</b><br>Score: %{x:.2f}<br>Empresas: %{customdata[0]:,}<extra></extra>',
            customdata=df_setor[['Qtd Empresas']].values
        )
    ])
    
    fig.update_layout(
        title='<b>Score de Risco Médio por Setor (Top 20)</b>',
        xaxis_title='Score Médio',
        yaxis_title='Setor',
        height=700,
        yaxis=dict(autorange='reversed')
    )
    
    fig.show()
    
    # ========================================================================
    # EMPRESAS DE ALTO RISCO
    # ========================================================================
    
    print("\n" + "="*80)
    print("🚨 EMPRESAS DE ALTO RISCO (Top 30)")
    print("="*80)
    
    df_alto_risco = df_score[
        df_score['classificacao_risco'].isin(['RISCO CRÍTICO', 'RISCO ALTO'])
    ].sort_values('score_risco_total', ascending=False).head(30)
    
    print(f"\n✅ {len(df_alto_risco)} empresas de alto risco identificadas")
    
    if len(df_alto_risco) > 0:
        print("\n📋 Resumo das Top 30:")
        print(df_alto_risco[[
            'razao_social', 'uf', 'classificacao_risco', 
            'score_risco_total', 'qtd_indicios'
        ]].to_string(index=False))
        
        # Gráfico das top 30
        df_alto_risco['razao_curta'] = df_alto_risco['razao_social'].str[:40]
        
        fig = go.Figure(data=[
            go.Bar(
                y=df_alto_risco['razao_curta'],
                x=df_alto_risco['score_risco_total'],
                orientation='h',
                marker=dict(color='#d62728'),
                text=df_alto_risco['score_risco_total'].round(2),
                textposition='outside',
                hovertemplate='<b>%{y}</b><br>Score: %{x:.2f}<br>Indícios: %{customdata[0]:.0f}<extra></extra>',
                customdata=df_alto_risco[['qtd_indicios']].values
            )
        ])
        
        fig.update_layout(
            title='<b>Top 30 Empresas - Maior Score de Risco</b>',
            xaxis_title='Score de Risco Total',
            yaxis_title='Empresa',
            height=900,
            yaxis=dict(autorange='reversed')
        )
        
        fig.show()

else:
    print("\n⚠️ Dados de score de risco não disponíveis")

print("\n✅ Análise de risco concluída!")

In [ ]:
# ============================================================================
# CÉLULA 6: MACHINE LEARNING - RANDOM FOREST E XGBOOST
# ============================================================================

print("\n" + "="*80)
print("🤖 MACHINE LEARNING - PREDIÇÃO DE RISCO")
print("="*80)

# ============================================================================
# PREPARAR DADOS PARA ML
# ============================================================================

spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW vw_ml_dataset AS
SELECT 
    sr.cnpj,
    sr.classificacao_risco,
    CAST(COALESCE(ind.ativo_total, 0) AS DOUBLE) AS ativo_total,
    CAST(COALESCE(ind.ativo_circulante, 0) AS DOUBLE) AS ativo_circulante,
    CAST(COALESCE(ind.passivo_circulante, 0) AS DOUBLE) AS passivo_circulante,
    CAST(COALESCE(ind.patrimonio_liquido, 0) AS DOUBLE) AS patrimonio_liquido,
    CAST(COALESCE(ind.receita_liquida, 0) AS DOUBLE) AS receita_liquida,
    CAST(COALESCE(ind.resultado_liquido, 0) AS DOUBLE) AS resultado_liquido,
    CAST(COALESCE(ind.liquidez_corrente, 0) AS DOUBLE) AS liquidez_corrente,
    CAST(COALESCE(ind.liquidez_geral, 0) AS DOUBLE) AS liquidez_geral,
    CAST(COALESCE(ind.endividamento_geral, 0) AS DOUBLE) AS endividamento_geral,
    CAST(COALESCE(ind.margem_liquida_perc, 0) AS DOUBLE) AS margem_liquida_perc,
    CAST(COALESCE(ind.margem_bruta_perc, 0) AS DOUBLE) AS margem_bruta_perc,
    CAST(COALESCE(ind.roa_retorno_ativo_perc, 0) AS DOUBLE) AS roa_perc,
    CAST(COALESCE(ind.roe_retorno_patrimonio_perc, 0) AS DOUBLE) AS roe_perc,
    CAST(COALESCE(sr.score_equacao_contabil, 0) AS DOUBLE) AS score_equacao,
    CAST(COALESCE(sr.qtd_indicios_neaf, 0) AS DOUBLE) AS qtd_indicios,
    CASE 
        WHEN sr.empresa_grande_porte = 'Sim' THEN 1 
        ELSE 0 
    END AS eh_grande_porte
FROM teste.ecd_score_risco_consolidado sr
INNER JOIN teste.ecd_indicadores_financeiros ind
    ON sr.cnpj = ind.cnpj
    AND sr.ano_referencia = ind.ano_referencia
WHERE sr.classificacao_risco IS NOT NULL
    AND ind.ativo_total > 0
    AND ind.liquidez_corrente IS NOT NULL
""")

# Verificar tamanho
total_ml = spark.sql("SELECT COUNT(*) as cnt FROM vw_ml_dataset").collect()[0]['cnt']
print(f"\n📊 Total de registros para ML: {total_ml:,}")

if total_ml > 0 and total_ml <= 100000:
    df_ml = spark.sql("SELECT * FROM vw_ml_dataset").toPandas()
    
    print(f"✅ Dataset ML carregado: {len(df_ml):,} registros")
    
    # ========================================================================
    # PREPARAÇÃO DOS DADOS
    # ========================================================================
    
    print("\n" + "="*80)
    print("🔧 PREPARAÇÃO DOS DADOS")
    print("="*80)
    
    # Verificar distribuição das classes
    print("\n📊 Distribuição das classes:")
    print(df_ml['classificacao_risco'].value_counts())
    
    # Criar variável target binária (Alto Risco vs Baixo Risco)
    df_ml['target_binario'] = df_ml['classificacao_risco'].apply(
        lambda x: 1 if x in ['RISCO CRÍTICO', 'RISCO ALTO'] else 0
    )
    
    print(f"\n📊 Classes balanceadas:")
    print(f"  • Alto Risco: {df_ml['target_binario'].sum():,} ({df_ml['target_binario'].mean()*100:.1f}%)")
    print(f"  • Baixo Risco: {(df_ml['target_binario']==0).sum():,} ({(df_ml['target_binario']==0).mean()*100:.1f}%)")
    
    # Features para o modelo
    features = [
        'ativo_total', 'ativo_circulante', 'passivo_circulante', 'patrimonio_liquido',
        'receita_liquida', 'resultado_liquido', 'liquidez_corrente', 'liquidez_geral',
        'endividamento_geral', 'margem_liquida_perc', 'margem_bruta_perc',
        'roa_perc', 'roe_perc', 'score_equacao', 'qtd_indicios', 'eh_grande_porte'
    ]
    
    # Remover valores infinitos e NaN
    df_ml[features] = df_ml[features].replace([np.inf, -np.inf], np.nan)
    df_ml = df_ml.dropna(subset=features + ['target_binario'])
    
    print(f"\n✅ Dataset limpo: {len(df_ml):,} registros")
    
    # Separar features e target
    X = df_ml[features]
    y = df_ml['target_binario']
    
    # ========================================================================
    # NORMALIZAÇÃO
    # ========================================================================
    
    print("\n📊 Normalizando features...")
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = pd.DataFrame(X_scaled, columns=features, index=X.index)
    
    # Split train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.3, random_state=42, stratify=y
    )
    
    print(f"✅ Train set: {len(X_train):,} | Test set: {len(X_test):,}")
    
    # ========================================================================
    # MODELO 1: RANDOM FOREST
    # ========================================================================
    
    print("\n" + "="*80)
    print("🌲 RANDOM FOREST CLASSIFIER")
    print("="*80)
    
    print("\n⏳ Treinando Random Forest...")
    rf_model = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        min_samples_split=20,
        min_samples_leaf=10,
        random_state=42,
        n_jobs=-1,
        class_weight='balanced'
    )
    
    rf_model.fit(X_train, y_train)
    
    # Predições
    y_pred_rf = rf_model.predict(X_test)
    y_proba_rf = rf_model.predict_proba(X_test)[:, 1]
    
    # Métricas
    print("\n📊 RESULTADOS RANDOM FOREST:")
    print("\n" + classification_report(y_test, y_pred_rf, target_names=['Baixo Risco', 'Alto Risco']))
    
    # ROC AUC
    roc_auc_rf = roc_auc_score(y_test, y_proba_rf)
    print(f"\n🎯 ROC AUC Score: {roc_auc_rf:.4f}")
    
    # Feature Importance
    feature_importance_rf = pd.DataFrame({
        'feature': features,
        'importance': rf_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\n📊 TOP 10 FEATURES MAIS IMPORTANTES (Random Forest):")
    print(feature_importance_rf.head(10).to_string(index=False))
    
    # Gráfico de importância
    fig = go.Figure(data=[
        go.Bar(
            y=feature_importance_rf.head(15)['feature'],
            x=feature_importance_rf.head(15)['importance'],
            orientation='h',
            marker=dict(color='#2ca02c'),
            text=feature_importance_rf.head(15)['importance'].round(3),
            textposition='outside'
        )
    ])
    
    fig.update_layout(
        title='<b>Feature Importance - Random Forest</b>',
        xaxis_title='Importância',
        yaxis_title='Feature',
        height=600,
        yaxis=dict(autorange='reversed')
    )
    
    fig.show()
    
    # ========================================================================
    # MODELO 2: XGBOOST
    # ========================================================================
    
    print("\n" + "="*80)
    print("⚡ XGBOOST CLASSIFIER")
    print("="*80)
    
    print("\n⏳ Treinando XGBoost...")
    
    # Calcular scale_pos_weight para balancear classes
    scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
    
    xgb_model = xgb.XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        scale_pos_weight=scale_pos_weight,
        random_state=42,
        n_jobs=-1,
        eval_metric='logloss'
    )
    
    xgb_model.fit(X_train, y_train, verbose=False)
    
    # Predições
    y_pred_xgb = xgb_model.predict(X_test)
    y_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]
    
    # Métricas
    print("\n📊 RESULTADOS XGBOOST:")
    print("\n" + classification_report(y_test, y_pred_xgb, target_names=['Baixo Risco', 'Alto Risco']))
    
    # ROC AUC
    roc_auc_xgb = roc_auc_score(y_test, y_proba_xgb)
    print(f"\n🎯 ROC AUC Score: {roc_auc_xgb:.4f}")
    
    # Feature Importance
    feature_importance_xgb = pd.DataFrame({
        'feature': features,
        'importance': xgb_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\n📊 TOP 10 FEATURES MAIS IMPORTANTES (XGBoost):")
    print(feature_importance_xgb.head(10).to_string(index=False))
    
    # Gráfico de importância
    fig = go.Figure(data=[
        go.Bar(
            y=feature_importance_xgb.head(15)['feature'],
            x=feature_importance_xgb.head(15)['importance'],
            orientation='h',
            marker=dict(color='#ff7f0e'),
            text=feature_importance_xgb.head(15)['importance'].round(3),
            textposition='outside'
        )
    ])
    
    fig.update_layout(
        title='<b>Feature Importance - XGBoost</b>',
        xaxis_title='Importância',
        yaxis_title='Feature',
        height=600,
        yaxis=dict(autorange='reversed')
    )
    
    fig.show()
    
    # ========================================================================
    # COMPARAÇÃO DOS MODELOS
    # ========================================================================
    
    print("\n" + "="*80)
    print("🏆 COMPARAÇÃO DOS MODELOS")
    print("="*80)
    
    comparacao = pd.DataFrame({
        'Modelo': ['Random Forest', 'XGBoost'],
        'ROC AUC': [roc_auc_rf, roc_auc_xgb],
        'Acurácia': [
            (y_test == y_pred_rf).mean(),
            (y_test == y_pred_xgb).mean()
        ]
    })
    
    print("\n" + comparacao.to_string(index=False))
    
    # Gráfico de comparação
    fig = go.Figure(data=[
        go.Bar(name='ROC AUC', x=comparacao['Modelo'], y=comparacao['ROC AUC'], marker=dict(color='#1f77b4')),
        go.Bar(name='Acurácia', x=comparacao['Modelo'], y=comparacao['Acurácia'], marker=dict(color='#ff7f0e'))
    ])
    
    fig.update_layout(
        title='<b>Comparação de Performance dos Modelos</b>',
        xaxis_title='Modelo',
        yaxis_title='Score',
        barmode='group',
        height=400,
        yaxis=dict(range=[0, 1])
    )
    
    fig.show()
    
    # ========================================================================
    # MATRIZ DE CONFUSÃO
    # ========================================================================
    
    print("\n" + "="*80)
    print("📊 MATRIZ DE CONFUSÃO")
    print("="*80)
    
    # Matriz Random Forest
    cm_rf = confusion_matrix(y_test, y_pred_rf)
    cm_xgb = confusion_matrix(y_test, y_pred_xgb)
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Random Forest', 'XGBoost'),
        specs=[[{'type': 'heatmap'}, {'type': 'heatmap'}]]
    )
    
    fig.add_trace(
        go.Heatmap(
            z=cm_rf,
            x=['Predito: Baixo', 'Predito: Alto'],
            y=['Real: Baixo', 'Real: Alto'],
            colorscale='Blues',
            text=cm_rf,
            texttemplate='%{text}',
            showscale=False
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Heatmap(
            z=cm_xgb,
            x=['Predito: Baixo', 'Predito: Alto'],
            y=['Real: Baixo', 'Real: Alto'],
            colorscale='Oranges',
            text=cm_xgb,
            texttemplate='%{text}',
            showscale=False
        ),
        row=1, col=2
    )
    
    fig.update_layout(
        title='<b>Matriz de Confusão - Comparação</b>',
        height=500
    )
    
    fig.show()

elif total_ml > 100000:
    print(f"\n⚠️ Dataset muito grande ({total_ml:,} registros). Limitando a 100.000 para ML...")
    df_ml = spark.sql("SELECT * FROM vw_ml_dataset LIMIT 100000").toPandas()
    print("Execute novamente a célula para treinar os modelos.")

else:
    print("\n⚠️ Dados insuficientes para Machine Learning")

print("\n✅ Análise de Machine Learning concluída!")

In [ ]:
# ============================================================================
# CÉLULA 7: CLUSTERING E APRENDIZADO NÃO SUPERVISIONADO
# ============================================================================

print("\n" + "="*80)
print("🔍 ANÁLISE DE CLUSTERING - APRENDIZADO NÃO SUPERVISIONADO")
print("="*80)

# ============================================================================
# PREPARAR DADOS PARA CLUSTERING
# ============================================================================

spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW vw_clustering AS
SELECT 
    ind.cnpj,
    ec.nm_razao_social,
    ec.cd_uf,
    ec.cnae_secao_descricao,
    CAST(COALESCE(ind.ativo_total, 0) AS DOUBLE) AS ativo_total,
    CAST(COALESCE(ind.receita_liquida, 0) AS DOUBLE) AS receita_liquida,
    CAST(COALESCE(ind.resultado_liquido, 0) AS DOUBLE) AS resultado_liquido,
    CAST(COALESCE(ind.liquidez_corrente, 0) AS DOUBLE) AS liquidez_corrente,
    CAST(COALESCE(ind.liquidez_geral, 0) AS DOUBLE) AS liquidez_geral,
    CAST(COALESCE(ind.endividamento_geral, 0) AS DOUBLE) AS endividamento_geral,
    CAST(COALESCE(ind.margem_liquida_perc, 0) AS DOUBLE) AS margem_liquida_perc,
    CAST(COALESCE(ind.roa_retorno_ativo_perc, 0) AS DOUBLE) AS roa_perc,
    CAST(COALESCE(ind.roe_retorno_patrimonio_perc, 0) AS DOUBLE) AS roe_perc,
    CAST(COALESCE(sr.score_risco_total, 0) AS DOUBLE) AS score_risco
FROM teste.ecd_indicadores_financeiros ind
INNER JOIN teste.ecd_empresas_cadastro ec
    ON ind.cnpj = ec.cnpj
    AND ind.ano_referencia = ec.ano_referencia
LEFT JOIN teste.ecd_score_risco_consolidado sr
    ON ind.cnpj = sr.cnpj
    AND ind.ano_referencia = sr.ano_referencia
WHERE ind.ativo_total > 0
    AND ind.liquidez_corrente IS NOT NULL
    AND ind.liquidez_corrente > 0
    AND ind.liquidez_corrente < 10
    AND ind.endividamento_geral >= 0
    AND ind.endividamento_geral <= 2
""")

# Verificar tamanho
total_cluster = spark.sql("SELECT COUNT(*) as cnt FROM vw_clustering").collect()[0]['cnt']
print(f"\n📊 Total de registros para clustering: {total_cluster:,}")

if total_cluster > 0:
    # Limitar a 20.000 registros
    if total_cluster > 20000:
        print(f"⚠️ Muitos registros ({total_cluster:,}), amostrando 20.000...")
        df_cluster_full = spark.sql("SELECT * FROM vw_clustering ORDER BY ativo_total DESC LIMIT 20000").toPandas()
    else:
        df_cluster_full = spark.sql("SELECT * FROM vw_clustering").toPandas()
    
    print(f"✅ Dataset carregado: {len(df_cluster_full):,} registros")
    
    # ========================================================================
    # PREPARAÇÃO DOS DADOS
    # ========================================================================
    
    print("\n" + "="*80)
    print("🔧 PREPARAÇÃO PARA CLUSTERING")
    print("="*80)
    
    # Features para clustering
    features_cluster = [
        'ativo_total', 'receita_liquida', 'resultado_liquido',
        'liquidez_corrente', 'liquidez_geral', 'endividamento_geral',
        'margem_liquida_perc', 'roa_perc', 'roe_perc', 'score_risco'
    ]
    
    # Remover NaN e infinitos
    df_cluster = df_cluster_full[['cnpj', 'nm_razao_social', 'cd_uf', 'cnae_secao_descricao'] + features_cluster].copy()
    df_cluster[features_cluster] = df_cluster[features_cluster].replace([np.inf, -np.inf], np.nan)
    df_cluster = df_cluster.dropna(subset=features_cluster)
    
    print(f"\n✅ Dataset limpo: {len(df_cluster):,} registros")
    
    # Separar features
    X_cluster = df_cluster[features_cluster]
    
    # Normalização
    print("\n📊 Normalizando features...")
    scaler_cluster = StandardScaler()
    X_scaled_cluster = scaler_cluster.fit_transform(X_cluster)
    X_scaled_df = pd.DataFrame(X_scaled_cluster, columns=features_cluster, index=X_cluster.index)
    
    # ========================================================================
    # MÉTODO DO COTOVELO (ELBOW METHOD)
    # ========================================================================
    
    print("\n" + "="*80)
    print("📊 MÉTODO DO COTOVELO - DETERMINANDO K ÓTIMO")
    print("="*80)
    
    # Testar de 2 a 10 clusters
    inertias = []
    silhouette_scores = []
    K_range = range(2, 11)
    
    print("\n⏳ Testando diferentes valores de K...")
    
    for k in K_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(X_scaled_cluster)
        inertias.append(kmeans.inertia_)
        silhouette_scores.append(silhouette_score(X_scaled_cluster, kmeans.labels_))
        print(f"  K={k}: Inertia={kmeans.inertia_:.2f}, Silhouette={silhouette_scores[-1]:.4f}")
    
    # Gráfico do cotovelo
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Método do Cotovelo', 'Silhouette Score')
    )
    
    fig.add_trace(
        go.Scatter(
            x=list(K_range),
            y=inertias,
            mode='lines+markers',
            marker=dict(size=10, color='#1f77b4'),
            line=dict(width=2)
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=list(K_range),
            y=silhouette_scores,
            mode='lines+markers',
            marker=dict(size=10, color='#ff7f0e'),
            line=dict(width=2)
        ),
        row=1, col=2
    )
    
    fig.update_xaxes(title_text="Número de Clusters (K)", row=1, col=1)
    fig.update_xaxes(title_text="Número de Clusters (K)", row=1, col=2)
    fig.update_yaxes(title_text="Inertia", row=1, col=1)
    fig.update_yaxes(title_text="Silhouette Score", row=1, col=2)
    
    fig.update_layout(
        title='<b>Determinação do Número Ótimo de Clusters</b>',
        height=400,
        showlegend=False
    )
    
    fig.show()
    
    # Escolher K ótimo (maior silhouette)
    k_otimo = list(K_range)[np.argmax(silhouette_scores)]
    silhouette_maximo = silhouette_scores[np.argmax(silhouette_scores)]
    print(f"\n✅ K ótimo sugerido: {k_otimo} (Silhouette Score: {silhouette_maximo:.4f})")
    
    # ========================================================================
    # K-MEANS CLUSTERING
    # ========================================================================
    
    print("\n" + "="*80)
    print(f"🎯 K-MEANS CLUSTERING (K={k_otimo})")
    print("="*80)
    
    print(f"\n⏳ Aplicando K-Means com {k_otimo} clusters...")
    kmeans_final = KMeans(n_clusters=k_otimo, random_state=42, n_init=20)
    df_cluster['cluster_kmeans'] = kmeans_final.fit_predict(X_scaled_cluster)
    
    # Métricas
    silhouette_avg = silhouette_score(X_scaled_cluster, df_cluster['cluster_kmeans'])
    davies_bouldin = davies_bouldin_score(X_scaled_cluster, df_cluster['cluster_kmeans'])
    calinski_harabasz = calinski_harabasz_score(X_scaled_cluster, df_cluster['cluster_kmeans'])
    
    print(f"\n📊 MÉTRICAS DE QUALIDADE:")
    print(f"  • Silhouette Score: {silhouette_avg:.4f} (quanto maior, melhor)")
    print(f"  • Davies-Bouldin Index: {davies_bouldin:.4f} (quanto menor, melhor)")
    print(f"  • Calinski-Harabasz Score: {calinski_harabasz:.2f} (quanto maior, melhor)")
    
    # Estatísticas por cluster
    print("\n" + "="*80)
    print("📊 PERFIL DOS CLUSTERS")
    print("="*80)
    
    for cluster_id in range(k_otimo):
        cluster_data = df_cluster[df_cluster['cluster_kmeans'] == cluster_id]
        print(f"\n🎯 CLUSTER {cluster_id} ({len(cluster_data):,} empresas - {len(cluster_data)/len(df_cluster)*100:.1f}%):")
        
        # Características médias
        print(f"  • Ativo Total Médio: R$ {cluster_data['ativo_total'].mean()/1e6:.2f}M")
        print(f"  • Receita Líquida Média: R$ {cluster_data['receita_liquida'].mean()/1e6:.2f}M")
        print(f"  • Liquidez Corrente Média: {cluster_data['liquidez_corrente'].mean():.2f}")
        print(f"  • Endividamento Médio: {cluster_data['endividamento_geral'].mean():.2%}")
        print(f"  • Margem Líquida Média: {cluster_data['margem_liquida_perc'].mean():.2f}%")
        print(f"  • Score de Risco Médio: {cluster_data['score_risco'].mean():.2f}")
    
    # ========================================================================
    # REDUÇÃO DE DIMENSIONALIDADE (PCA)
    # ========================================================================
    
    print("\n" + "="*80)
    print("🔬 REDUÇÃO DE DIMENSIONALIDADE - PCA")
    print("="*80)
    
    print("\n⏳ Aplicando PCA...")
    pca = PCA(n_components=2, random_state=42)
    X_pca = pca.fit_transform(X_scaled_cluster)
    
    df_cluster['pca1'] = X_pca[:, 0]
    df_cluster['pca2'] = X_pca[:, 1]
    
    variance_explained = pca.explained_variance_ratio_
    print(f"\n📊 Variância explicada:")
    print(f"  • PC1: {variance_explained[0]:.2%}")
    print(f"  • PC2: {variance_explained[1]:.2%}")
    # CORREÇÃO: Usar np.sum() em vez de sum() do PySpark
    print(f"  • Total: {np.sum(variance_explained):.2%}")
    
    # Visualização dos clusters no espaço PCA
    fig = px.scatter(
        df_cluster,
        x='pca1',
        y='pca2',
        color='cluster_kmeans',
        hover_data=['nm_razao_social', 'cd_uf', 'ativo_total', 'score_risco'],
        title='<b>Clusters K-Means - Visualização PCA</b>',
        labels={
            'pca1': f'PC1 ({variance_explained[0]:.1%} variância)',
            'pca2': f'PC2 ({variance_explained[1]:.1%} variância)',
            'cluster_kmeans': 'Cluster'
        },
        color_continuous_scale='Viridis'
    )
    
    fig.update_layout(height=600)
    fig.show()
    
    # ========================================================================
    # GRÁFICO: CARACTERÍSTICAS DOS CLUSTERS
    # ========================================================================
    
    print("\n" + "="*80)
    print("📊 VISUALIZAÇÃO: CARACTERÍSTICAS POR CLUSTER")
    print("="*80)
    
    # Preparar dados agregados
    cluster_stats = df_cluster.groupby('cluster_kmeans')[features_cluster].mean()
    cluster_stats_normalized = (cluster_stats - cluster_stats.min()) / (cluster_stats.max() - cluster_stats.min())
    
    # Heatmap
    fig = go.Figure(data=go.Heatmap(
        z=cluster_stats_normalized.values.T,
        x=[f'Cluster {i}' for i in range(k_otimo)],
        y=features_cluster,
        colorscale='RdYlGn',
        text=cluster_stats.values.T.round(2),
        texttemplate='%{text}',
        textfont={"size": 10},
        colorbar=dict(title="Valor Normalizado")
    ))
    
    fig.update_layout(
        title='<b>Perfil dos Clusters - Características Normalizadas</b>',
        xaxis_title='Cluster',
        yaxis_title='Feature',
        height=600
    )
    
    fig.show()
    
    # ========================================================================
    # DISTRIBUIÇÃO GEOGRÁFICA DOS CLUSTERS
    # ========================================================================
    
    print("\n" + "="*80)
    print("🗺️ DISTRIBUIÇÃO GEOGRÁFICA DOS CLUSTERS")
    print("="*80)
    
    cluster_uf = df_cluster.groupby(['cluster_kmeans', 'cd_uf']).size().reset_index(name='count')
    
    fig = px.bar(
        cluster_uf,
        x='cd_uf',
        y='count',
        color='cluster_kmeans',
        title='<b>Distribuição dos Clusters por UF</b>',
        labels={'count': 'Quantidade de Empresas', 'cd_uf': 'UF', 'cluster_kmeans': 'Cluster'},
        barmode='group'
    )
    
    fig.update_layout(height=500)
    fig.show()
    
    # ========================================================================
    # DBSCAN (OPCIONAL)
    # ========================================================================
    
    print("\n" + "="*80)
    print("🔍 DBSCAN - CLUSTERING BASEADO EM DENSIDADE")
    print("="*80)
    
    # Usar amostra menor para DBSCAN se dataset muito grande
    if len(df_cluster) > 5000:
        print(f"⚠️ Dataset muito grande, usando amostra de 5.000 registros...")
        sample_indices = np.random.choice(len(df_cluster), 5000, replace=False)
        X_dbscan = X_scaled_cluster[sample_indices]
        df_dbscan = df_cluster.iloc[sample_indices].copy()
    else:
        X_dbscan = X_scaled_cluster
        df_dbscan = df_cluster.copy()
    
    print("\n⏳ Aplicando DBSCAN...")
    dbscan = DBSCAN(eps=0.5, min_samples=10)
    df_dbscan['cluster_dbscan'] = dbscan.fit_predict(X_dbscan)
    
    n_clusters_dbscan = len(set(df_dbscan['cluster_dbscan'])) - (1 if -1 in df_dbscan['cluster_dbscan'] else 0)
    n_noise = list(df_dbscan['cluster_dbscan']).count(-1)
    
    print(f"\n📊 Clusters encontrados: {n_clusters_dbscan}")
    print(f"📊 Pontos de ruído: {n_noise} ({n_noise/len(df_dbscan)*100:.1f}%)")
    
    if n_clusters_dbscan > 0 and n_clusters_dbscan < 20:
        # Visualizar DBSCAN
        df_dbscan_plot = df_dbscan.copy()
        pca_dbscan = PCA(n_components=2, random_state=42)
        X_pca_dbscan = pca_dbscan.fit_transform(X_dbscan)
        df_dbscan_plot['pca1'] = X_pca_dbscan[:, 0]
        df_dbscan_plot['pca2'] = X_pca_dbscan[:, 1]
        
        fig = px.scatter(
            df_dbscan_plot,
            x='pca1',
            y='pca2',
            color='cluster_dbscan',
            title='<b>DBSCAN Clusters - Visualização PCA</b>',
            labels={'cluster_dbscan': 'Cluster (-1 = Ruído)'},
            color_continuous_scale='Viridis'
        )
        
        fig.update_layout(height=600)
        fig.show()

else:
    print("\n⚠️ Dados insuficientes para clustering")

print("\n✅ Análise de clustering concluída!")

In [ ]:
# ============================================================================
# CÉLULA 8: ANÁLISE SETORIAL E BENCHMARKING
# ============================================================================

print("\n" + "="*80)
print("🏭 ANÁLISE SETORIAL E BENCHMARKING")
print("="*80)

# ============================================================================
# PREPARAR DADOS SETORIAIS
# ============================================================================

spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW vw_analise_setorial AS
SELECT 
    ec.cnae_secao,
    ec.cnae_secao_descricao,
    ec.cnae_divisao,
    ec.cnae_divisao_descricao,
    CAST(ind.ano_referencia / 100 AS INT) AS ano_fiscal,
    COUNT(DISTINCT ec.cnpj) AS qtd_empresas,
    CAST(COALESCE(AVG(ind.ativo_total), 0) AS DOUBLE) AS media_ativo,
    CAST(COALESCE(AVG(ind.receita_liquida), 0) AS DOUBLE) AS media_receita,
    CAST(COALESCE(AVG(ind.resultado_liquido), 0) AS DOUBLE) AS media_resultado,
    CAST(COALESCE(AVG(ind.liquidez_corrente), 0) AS DOUBLE) AS media_liquidez,
    CAST(COALESCE(AVG(ind.endividamento_geral), 0) AS DOUBLE) AS media_endividamento,
    CAST(COALESCE(AVG(ind.margem_liquida_perc), 0) AS DOUBLE) AS media_margem_liquida,
    CAST(COALESCE(AVG(ind.roe_retorno_patrimonio_perc), 0) AS DOUBLE) AS media_roe,
    CAST(COALESCE(AVG(sr.score_risco_total), 0) AS DOUBLE) AS media_score_risco,
    COUNT(DISTINCT CASE WHEN ec.empresa_grande_porte = 'Sim' THEN ec.cnpj END) AS qtd_grande_porte,
    COUNT(DISTINCT CASE WHEN sr.classificacao_risco IN ('RISCO CRÍTICO', 'RISCO ALTO') THEN ec.cnpj END) AS qtd_alto_risco
FROM teste.ecd_empresas_cadastro ec
INNER JOIN teste.ecd_indicadores_financeiros ind
    ON ec.cnpj = ind.cnpj
    AND ec.ano_referencia = ind.ano_referencia
LEFT JOIN teste.ecd_score_risco_consolidado sr
    ON ec.cnpj = sr.cnpj
    AND ec.ano_referencia = sr.ano_referencia
WHERE ec.cnae_secao IS NOT NULL
    AND ind.ativo_total > 0
GROUP BY ec.cnae_secao, ec.cnae_secao_descricao, ec.cnae_divisao, ec.cnae_divisao_descricao,
         CAST(ind.ano_referencia / 100 AS INT)
HAVING COUNT(DISTINCT ec.cnpj) >= 3
""")

# Verificar tamanho
total_setorial = spark.sql("SELECT COUNT(*) as cnt FROM vw_analise_setorial").collect()[0]['cnt']
print(f"\n📊 Total de setores/períodos: {total_setorial:,}")

if total_setorial > 0:
    # Limitar se necessário
    if total_setorial > 1000:
        print(f"⚠️ Muitos registros ({total_setorial:,}), limitando a 1.000...")
        df_setorial = spark.sql("SELECT * FROM vw_analise_setorial ORDER BY qtd_empresas DESC LIMIT 1000").toPandas()
    else:
        df_spark = spark.sql("SELECT * FROM vw_analise_setorial")
        df_spark.cache()
        df_setorial = df_spark.toPandas()
    
    print(f"✅ Dados setoriais carregados: {len(df_setorial):,} registros")
    
    # ========================================================================
    # TOP SETORES POR QUANTIDADE DE EMPRESAS
    # ========================================================================
    
    print("\n" + "="*80)
    print("📊 TOP 20 SETORES - QUANTIDADE DE EMPRESAS")
    print("="*80)
    
    # Agregar por seção (ano mais recente)
    ano_mais_recente = df_setorial['ano_fiscal'].max()
    df_ano_atual = df_setorial[df_setorial['ano_fiscal'] == ano_mais_recente].copy()
    
    df_top_setores = df_ano_atual.groupby('cnae_secao_descricao').agg({
        'qtd_empresas': 'sum',
        'media_ativo': 'mean',
        'media_receita': 'mean',
        'media_score_risco': 'mean',
        'qtd_grande_porte': 'sum',
        'qtd_alto_risco': 'sum'
    }).reset_index()
    
    df_top_setores = df_top_setores.sort_values('qtd_empresas', ascending=False).head(20)
    df_top_setores['setor_curto'] = df_top_setores['cnae_secao_descricao'].str[:50]
    
    print(f"\n📅 Ano de referência: {ano_mais_recente}")
    print("\n" + df_top_setores[['setor_curto', 'qtd_empresas', 'media_ativo', 'media_receita']].to_string(index=False))
    
    # Gráfico Top Setores
    fig = go.Figure(data=[
        go.Bar(
            y=df_top_setores['setor_curto'],
            x=df_top_setores['qtd_empresas'],
            orientation='h',
            marker=dict(
                color=df_top_setores['qtd_empresas'],
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="Empresas")
            ),
            text=df_top_setores['qtd_empresas'],
            textposition='outside',
            hovertemplate='<b>%{y}</b><br>Empresas: %{x:,}<extra></extra>'
        )
    ])
    
    fig.update_layout(
        title='<b>Top 20 Setores por Quantidade de Empresas</b>',
        xaxis_title='Quantidade de Empresas',
        yaxis_title='Setor',
        height=700,
        yaxis=dict(autorange='reversed')
    )
    
    fig.show()
    
    # ========================================================================
    # COMPARAÇÃO DE INDICADORES POR SETOR
    # ========================================================================
    
    print("\n" + "="*80)
    print("📊 COMPARAÇÃO DE INDICADORES - TOP 15 SETORES")
    print("="*80)
    
    df_top15 = df_top_setores.head(15).copy()
    df_top15['ativo_milhoes'] = df_top15['media_ativo'] / 1_000_000
    df_top15['receita_milhoes'] = df_top15['media_receita'] / 1_000_000
    
    # Gráfico de comparação múltipla
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Ativo Médio (Milhões R$)',
            'Receita Média (Milhões R$)',
            'Score de Risco Médio',
            'Liquidez Corrente Média'
        ),
        vertical_spacing=0.15,
        horizontal_spacing=0.12
    )
    
    # Ativo
    fig.add_trace(
        go.Bar(
            y=df_top15['setor_curto'],
            x=df_top15['ativo_milhoes'],
            orientation='h',
            marker=dict(color='#1f77b4'),
            showlegend=False
        ),
        row=1, col=1
    )
    
    # Receita
    fig.add_trace(
        go.Bar(
            y=df_top15['setor_curto'],
            x=df_top15['receita_milhoes'],
            orientation='h',
            marker=dict(color='#ff7f0e'),
            showlegend=False
        ),
        row=1, col=2
    )
    
    # Score de Risco
    fig.add_trace(
        go.Bar(
            y=df_top15['setor_curto'],
            x=df_top15['media_score_risco'],
            orientation='h',
            marker=dict(color='#d62728'),
            showlegend=False
        ),
        row=2, col=1
    )
    
    # Liquidez
    fig.add_trace(
        go.Bar(
            y=df_top15['setor_curto'],
            x=df_ano_atual.groupby('cnae_secao_descricao')['media_liquidez'].mean().loc[df_top15['cnae_secao_descricao']].values,
            orientation='h',
            marker=dict(color='#2ca02c'),
            showlegend=False
        ),
        row=2, col=2
    )
    
    # Atualizar layout
    for i in range(1, 3):
        for j in range(1, 3):
            fig.update_yaxes(autorange='reversed', row=i, col=j)
    
    fig.update_layout(
        title='<b>Comparação de Indicadores por Setor</b>',
        height=900,
        showlegend=False
    )
    
    fig.show()
    
    # ========================================================================
    # SETORES COM MAIOR RISCO
    # ========================================================================
    
    print("\n" + "="*80)
    print("⚠️ SETORES COM MAIOR RISCO")
    print("="*80)
    
    df_risco_setor = df_ano_atual.sort_values('media_score_risco', ascending=False).head(15).copy()
    df_risco_setor['perc_alto_risco'] = (df_risco_setor['qtd_alto_risco'] / df_risco_setor['qtd_empresas']) * 100
    
    print("\n📊 Top 15 setores com maior score de risco:")
    print(df_risco_setor[['cnae_secao_descricao', 'qtd_empresas', 'media_score_risco', 'perc_alto_risco']].to_string(index=False))
    
    # Gráfico
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        y=df_risco_setor['cnae_secao_descricao'].str[:40],
        x=df_risco_setor['media_score_risco'],
        name='Score Médio',
        orientation='h',
        marker=dict(color='#d62728'),
        text=df_risco_setor['media_score_risco'].round(2),
        textposition='outside'
    ))
    
    fig.update_layout(
        title='<b>Top 15 Setores - Maior Score de Risco</b>',
        xaxis_title='Score de Risco Médio',
        yaxis_title='Setor',
        height=600,
        yaxis=dict(autorange='reversed')
    )
    
    fig.show()
    
    # ========================================================================
    # ANÁLISE DE RENTABILIDADE POR SETOR
    # ========================================================================
    
    print("\n" + "="*80)
    print("💹 ANÁLISE DE RENTABILIDADE POR SETOR")
    print("="*80)
    
    df_rentabilidade = df_ano_atual[df_ano_atual['qtd_empresas'] >= 10].sort_values('media_margem_liquida', ascending=False).head(15).copy()
    
    print("\n📊 Top 15 setores - Maior margem líquida:")
    print(df_rentabilidade[['cnae_secao_descricao', 'qtd_empresas', 'media_margem_liquida', 'media_roe']].to_string(index=False))
    
    # Scatter: Margem x ROE
    fig = px.scatter(
        df_ano_atual[df_ano_atual['qtd_empresas'] >= 5],
        x='media_margem_liquida',
        y='media_roe',
        size='qtd_empresas',
        color='media_score_risco',
        hover_data=['cnae_secao_descricao', 'qtd_empresas'],
        labels={
            'media_margem_liquida': 'Margem Líquida Média (%)',
            'media_roe': 'ROE Médio (%)',
            'qtd_empresas': 'Qtd. Empresas',
            'media_score_risco': 'Score de Risco'
        },
        title='<b>Rentabilidade por Setor: Margem Líquida vs ROE</b>',
        color_continuous_scale='RdYlGn_r'
    )
    
    # Adicionar linhas de referência
    fig.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.5)
    fig.add_vline(x=0, line_dash="dash", line_color="gray", opacity=0.5)
    
    fig.update_layout(height=600)
    fig.show()
    
    # ========================================================================
    # EVOLUÇÃO TEMPORAL POR SETOR (TOP 5)
    # ========================================================================
    
    if df_setorial['ano_fiscal'].nunique() > 1:
        print("\n" + "="*80)
        print("📈 EVOLUÇÃO TEMPORAL - TOP 5 SETORES")
        print("="*80)
        
        # Identificar top 5 setores
        top5_setores = df_ano_atual.nlargest(5, 'qtd_empresas')['cnae_secao_descricao'].tolist()
        df_evolucao = df_setorial[df_setorial['cnae_secao_descricao'].isin(top5_setores)].copy()
        
        # Agregar por ano e setor
        df_evolucao_agg = df_evolucao.groupby(['ano_fiscal', 'cnae_secao_descricao']).agg({
            'qtd_empresas': 'sum',
            'media_receita': 'mean',
            'media_score_risco': 'mean'
        }).reset_index()
        
        # Gráfico de evolução
        fig = make_subplots(
            rows=2, cols=1,
            subplot_titles=('Evolução da Quantidade de Empresas', 'Evolução do Score de Risco'),
            vertical_spacing=0.15
        )
        
        for setor in top5_setores:
            dados_setor = df_evolucao_agg[df_evolucao_agg['cnae_secao_descricao'] == setor]
            setor_curto = setor[:30]
            
            fig.add_trace(
                go.Scatter(
                    x=dados_setor['ano_fiscal'],
                    y=dados_setor['qtd_empresas'],
                    name=setor_curto,
                    mode='lines+markers',
                    line=dict(width=2),
                    marker=dict(size=8)
                ),
                row=1, col=1
            )
            
            fig.add_trace(
                go.Scatter(
                    x=dados_setor['ano_fiscal'],
                    y=dados_setor['media_score_risco'],
                    name=setor_curto,
                    mode='lines+markers',
                    line=dict(width=2),
                    marker=dict(size=8),
                    showlegend=False
                ),
                row=2, col=1
            )
        
        fig.update_xaxes(title_text="Ano Fiscal", row=2, col=1)
        fig.update_yaxes(title_text="Quantidade de Empresas", row=1, col=1)
        fig.update_yaxes(title_text="Score de Risco Médio", row=2, col=1)
        
        fig.update_layout(
            title='<b>Evolução Temporal - Top 5 Setores</b>',
            height=700,
            hovermode='x unified'
        )
        
        fig.show()
    
    # ========================================================================
    # MATRIZ DE COMPARAÇÃO SETORIAL
    # ========================================================================
    
    print("\n" + "="*80)
    print("📊 MATRIZ DE COMPARAÇÃO SETORIAL")
    print("="*80)
    
    # Selecionar top 10 setores
    top10_setores = df_top_setores.head(10)['cnae_secao_descricao'].tolist()
    df_matriz = df_ano_atual[df_ano_atual['cnae_secao_descricao'].isin(top10_setores)].copy()
    
    # Normalizar valores para comparação
    features_comparacao = ['media_ativo', 'media_receita', 'media_liquidez', 'media_endividamento', 
                           'media_margem_liquida', 'media_roe', 'media_score_risco']
    
    df_matriz_norm = df_matriz.set_index('cnae_secao_descricao')[features_comparacao].copy()
    df_matriz_norm = (df_matriz_norm - df_matriz_norm.min()) / (df_matriz_norm.max() - df_matriz_norm.min())
    
    # Preparar labels
    labels_curtos = [s[:30] for s in df_matriz_norm.index]
    
    # Heatmap
    fig = go.Figure(data=go.Heatmap(
        z=df_matriz_norm.values,
        x=['Ativo', 'Receita', 'Liquidez', 'Endiv.', 'Margem', 'ROE', 'Risco'],
        y=labels_curtos,
        colorscale='RdYlGn',
        text=df_matriz_norm.values.round(2),
        texttemplate='%{text}',
        textfont={"size": 9},
        colorbar=dict(title="Normalizado<br>(0-1)")
    ))
    
    fig.update_layout(
        title='<b>Matriz de Comparação Setorial (Valores Normalizados)</b>',
        xaxis_title='Indicador',
        yaxis_title='Setor',
        height=600
    )
    
    fig.show()

else:
    print("\n⚠️ Dados setoriais não disponíveis")

print("\n✅ Análise setorial concluída!")

In [ ]:
# ============================================================================
# CÉLULA 9: DASHBOARD EXECUTIVO INTERATIVO
# ============================================================================

print("\n" + "="*80)
print("📊 DASHBOARD EXECUTIVO - ECD ONLINE")
print("="*80)

# ============================================================================
# PREPARAR DADOS CONSOLIDADOS PARA DASHBOARD
# ============================================================================

spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW vw_dashboard AS
SELECT 
    ec.cnpj,
    ec.nm_razao_social,
    ec.cd_uf,
    ec.cnae_secao,
    ec.cnae_secao_descricao,
    ec.empresa_grande_porte,
    CAST(ind.ano_referencia / 100 AS INT) AS ano_fiscal,
    CAST(COALESCE(ind.ativo_total, 0) AS DOUBLE) AS ativo_total,
    CAST(COALESCE(ind.receita_liquida, 0) AS DOUBLE) AS receita_liquida,
    CAST(COALESCE(ind.resultado_liquido, 0) AS DOUBLE) AS resultado_liquido,
    CAST(COALESCE(ind.liquidez_corrente, 0) AS DOUBLE) AS liquidez_corrente,
    CAST(COALESCE(ind.endividamento_geral, 0) AS DOUBLE) AS endividamento_geral,
    CAST(COALESCE(ind.margem_liquida_perc, 0) AS DOUBLE) AS margem_liquida_perc,
    CAST(COALESCE(ind.roe_retorno_patrimonio_perc, 0) AS DOUBLE) AS roe_perc,
    sr.classificacao_risco,
    CAST(COALESCE(sr.score_risco_total, 0) AS DOUBLE) AS score_risco,
    CAST(COALESCE(sr.qtd_indicios_neaf, 0) AS DOUBLE) AS qtd_indicios
FROM teste.ecd_indicadores_financeiros ind
INNER JOIN teste.ecd_empresas_cadastro ec
    ON ind.cnpj = ec.cnpj
    AND ind.ano_referencia = ec.ano_referencia
LEFT JOIN teste.ecd_score_risco_consolidado sr
    ON ind.cnpj = sr.cnpj
    AND ind.ano_referencia = sr.ano_referencia
WHERE ind.ativo_total > 0
""")

# Verificar tamanho
total_dash = spark.sql("SELECT COUNT(*) as cnt FROM vw_dashboard").collect()[0]['cnt']
print(f"\n📊 Total de registros para dashboard: {total_dash:,}")

if total_dash > 0:
    # Limitar a 30.000 registros
    if total_dash > 30000:
        print(f"⚠️ Muitos registros ({total_dash:,}), limitando a 30.000...")
        df_dash = spark.sql("SELECT * FROM vw_dashboard ORDER BY ativo_total DESC LIMIT 30000").toPandas()
    else:
        df_spark = spark.sql("SELECT * FROM vw_dashboard")
        df_spark.cache()
        df_dash = df_spark.toPandas()
    
    print(f"✅ Dados do dashboard carregados: {len(df_dash):,} registros")
    
    # ========================================================================
    # MÉTRICAS PRINCIPAIS (KPIs)
    # ========================================================================
    
    print("\n" + "="*80)
    print("📊 INDICADORES-CHAVE (KPIs)")
    print("="*80)
    
    # Calcular KPIs
    kpis = {
        'total_empresas': df_dash['cnpj'].nunique(),
        'ativo_total': df_dash['ativo_total'].sum(),
        'receita_total': df_dash['receita_liquida'].sum(),
        'resultado_total': df_dash['resultado_liquido'].sum(),
        'empresas_lucro': (df_dash['resultado_liquido'] > 0).sum(),
        'empresas_prejuizo': (df_dash['resultado_liquido'] < 0).sum(),
        'empresas_alto_risco': df_dash[df_dash['classificacao_risco'].isin(['RISCO CRÍTICO', 'RISCO ALTO'])]['cnpj'].nunique() if 'classificacao_risco' in df_dash.columns else 0,
        'liquidez_media': df_dash['liquidez_corrente'].mean(),
        'endividamento_medio': df_dash['endividamento_geral'].mean(),
        'margem_media': df_dash['margem_liquida_perc'].mean()
    }
    
    print(f"\n🏢 Total de Empresas: {kpis['total_empresas']:,}")
    print(f"💰 Ativo Total: R$ {kpis['ativo_total']/1e9:.2f} bilhões")
    print(f"💵 Receita Total: R$ {kpis['receita_total']/1e9:.2f} bilhões")
    print(f"📈 Resultado Total: R$ {kpis['resultado_total']/1e9:.2f} bilhões")
    print(f"✅ Empresas com Lucro: {kpis['empresas_lucro']:,} ({kpis['empresas_lucro']/len(df_dash)*100:.1f}%)")
    print(f"❌ Empresas com Prejuízo: {kpis['empresas_prejuizo']:,} ({kpis['empresas_prejuizo']/len(df_dash)*100:.1f}%)")
    print(f"⚠️ Empresas Alto Risco: {kpis['empresas_alto_risco']:,}")
    print(f"📊 Liquidez Corrente Média: {kpis['liquidez_media']:.2f}")
    print(f"📊 Endividamento Médio: {kpis['endividamento_medio']:.2%}")
    print(f"📊 Margem Líquida Média: {kpis['margem_media']:.2f}%")
    
    # ========================================================================
    # DASHBOARD PRINCIPAL - 6 PAINÉIS
    # ========================================================================
    
    print("\n" + "="*80)
    print("📊 GERANDO DASHBOARD PRINCIPAL...")
    print("="*80)
    
    # Criar subplots
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=(
            '📊 Distribuição de Risco',
            '💰 Top 10 Empresas por Ativo',
            '🗺️ Distribuição por UF',
            '🏭 Top 10 Setores',
            '📈 Indicadores Financeiros',
            '⚠️ Empresas em Risco'
        ),
        specs=[
            [{'type': 'pie'}, {'type': 'bar'}],
            [{'type': 'bar'}, {'type': 'bar'}],
            [{'type': 'box'}, {'type': 'scatter'}]
        ],
        vertical_spacing=0.12,
        horizontal_spacing=0.10
    )
    
    # 1. DISTRIBUIÇÃO DE RISCO (Pie Chart)
    if 'classificacao_risco' in df_dash.columns:
        dist_risco = df_dash['classificacao_risco'].value_counts()
        fig.add_trace(
            go.Pie(
                labels=dist_risco.index,
                values=dist_risco.values,
                hole=0.4,
                marker=dict(colors=['#d62728', '#ff7f0e', '#ffbb33', '#2ca02c']),
                textinfo='label+percent'
            ),
            row=1, col=1
        )
    
    # 2. TOP 10 EMPRESAS POR ATIVO
    df_top10_ativo = df_dash.nlargest(10, 'ativo_total').copy()
    df_top10_ativo['razao_curta'] = df_top10_ativo['nm_razao_social'].str[:25]
    df_top10_ativo['ativo_bi'] = df_top10_ativo['ativo_total'] / 1e9
    
    fig.add_trace(
        go.Bar(
            y=df_top10_ativo['razao_curta'],
            x=df_top10_ativo['ativo_bi'],
            orientation='h',
            marker=dict(color='#1f77b4'),
            showlegend=False
        ),
        row=1, col=2
    )
    
    # 3. DISTRIBUIÇÃO POR UF
    dist_uf = df_dash['cd_uf'].value_counts().head(15)
    
    fig.add_trace(
        go.Bar(
            x=dist_uf.index,
            y=dist_uf.values,
            marker=dict(color='#ff7f0e'),
            showlegend=False
        ),
        row=2, col=1
    )
    
    # 4. TOP 10 SETORES
    if 'cnae_secao_descricao' in df_dash.columns:
        dist_setor = df_dash.groupby('cnae_secao_descricao').size().sort_values(ascending=False).head(10)
        setores_curtos = [s[:25] for s in dist_setor.index]
        
        fig.add_trace(
            go.Bar(
                y=setores_curtos,
                x=dist_setor.values,
                orientation='h',
                marker=dict(color='#2ca02c'),
                showlegend=False
            ),
            row=2, col=2
        )
    
    # 5. BOX PLOT - INDICADORES FINANCEIROS
    indicadores = ['liquidez_corrente', 'endividamento_geral', 'margem_liquida_perc']
    cores_box = ['#1f77b4', '#ff7f0e', '#2ca02c']
    
    for idx, ind in enumerate(indicadores):
        # Limpar outliers extremos para visualização
        dados_clean = df_dash[ind].clip(
            df_dash[ind].quantile(0.05),
            df_dash[ind].quantile(0.95)
        )
        
        fig.add_trace(
            go.Box(
                y=dados_clean,
                name=ind.replace('_', ' ').title(),
                marker=dict(color=cores_box[idx]),
                showlegend=False
            ),
            row=3, col=1
        )
    
    # 6. SCATTER - RISCO vs ATIVO
    if 'score_risco' in df_dash.columns:
        # Preparar dados para scatter
        df_scatter = df_dash[
            (df_dash['score_risco'] > 0) &
            (df_dash['ativo_total'] > 0)
        ].copy()
        
        # Amostrar se muitos dados - FIX: Usar min() do Python
        tamanho_amostra_scatter = 1000 if len(df_scatter) > 1000 else len(df_scatter)
        df_scatter = df_scatter.sample(n=tamanho_amostra_scatter, random_state=42)
        
        df_scatter['ativo_log'] = np.log10(df_scatter['ativo_total'])
        
        fig.add_trace(
            go.Scatter(
                x=df_scatter['score_risco'],
                y=df_scatter['ativo_log'],
                mode='markers',
                marker=dict(
                    size=6,
                    color=df_scatter['score_risco'],
                    colorscale='Reds',
                    showscale=False,
                    opacity=0.6
                ),
                showlegend=False
            ),
            row=3, col=2
        )
    
    # Atualizar layouts dos eixos
    fig.update_xaxes(title_text="Ativo (Bi R$)", row=1, col=2)
    fig.update_xaxes(title_text="UF", row=2, col=1)
    fig.update_xaxes(title_text="Qtd. Empresas", row=2, col=2)
    fig.update_xaxes(title_text="Score de Risco", row=3, col=2)
    
    fig.update_yaxes(title_text="Empresa", row=1, col=2, autorange='reversed')
    fig.update_yaxes(title_text="Qtd. Empresas", row=2, col=1)
    fig.update_yaxes(title_text="Setor", row=2, col=2, autorange='reversed')
    fig.update_yaxes(title_text="Valor", row=3, col=1)
    fig.update_yaxes(title_text="Log10(Ativo)", row=3, col=2)
    
    # Layout geral
    fig.update_layout(
        title={
            'text': '<b>DASHBOARD EXECUTIVO - ECD ONLINE</b><br><sub>Escrituração Contábil Digital - Análise Consolidada</sub>',
            'x': 0.5,
            'xanchor': 'center'
        },
        height=1200,
        showlegend=False
    )
    
    fig.show()
    
    # ========================================================================
    # DASHBOARD FINANCEIRO DETALHADO
    # ========================================================================
    
    print("\n" + "="*80)
    print("💰 DASHBOARD FINANCEIRO DETALHADO")
    print("="*80)
    
    # Preparar dados
    df_financeiro = df_dash[df_dash['receita_liquida'] > 0].copy()
    df_financeiro['lucro_prejuizo'] = df_financeiro['resultado_liquido'].apply(
        lambda x: 'Lucro' if x > 0 else 'Prejuízo'
    )
    
    # Criar dashboard financeiro
    fig_fin = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            '💵 Distribuição de Receita Líquida',
            '📊 Lucro vs Prejuízo',
            '📈 ROE por Porte de Empresa',
            '💰 Receita x Resultado'
        ),
        specs=[
            [{'type': 'histogram'}, {'type': 'bar'}],
            [{'type': 'box'}, {'type': 'scatter'}]
        ],
        vertical_spacing=0.15,
        horizontal_spacing=0.12
    )
    
    # 1. Histograma de Receita
    fig_fin.add_trace(
        go.Histogram(
            x=np.log10(df_financeiro['receita_liquida'].clip(lower=1)),
            nbinsx=50,
            marker=dict(color='#1f77b4'),
            showlegend=False
        ),
        row=1, col=1
    )
    
    # 2. Bar - Lucro vs Prejuízo
    dist_resultado = df_financeiro['lucro_prejuizo'].value_counts()
    
    fig_fin.add_trace(
        go.Bar(
            x=dist_resultado.index,
            y=dist_resultado.values,
            marker=dict(color=['#2ca02c', '#d62728']),
            text=dist_resultado.values,
            textposition='outside',
            showlegend=False
        ),
        row=1, col=2
    )
    
    # 3. Box Plot - ROE por Porte
    if 'empresa_grande_porte' in df_financeiro.columns:
        for porte in df_financeiro['empresa_grande_porte'].unique():
            dados_porte = df_financeiro[df_financeiro['empresa_grande_porte'] == porte]['roe_perc']
            dados_porte = dados_porte.clip(-50, 50)
            
            fig_fin.add_trace(
                go.Box(
                    y=dados_porte,
                    name=f"Porte: {porte}",
                    showlegend=False
                ),
                row=2, col=1
            )
    
    # 4. Scatter - Receita x Resultado - FIX: Usar min() do Python
    tamanho_scatter_fin = 1000 if len(df_financeiro) > 1000 else len(df_financeiro)
    df_scatter_fin = df_financeiro.sample(n=tamanho_scatter_fin, random_state=42).copy()
    df_scatter_fin['receita_mi'] = df_scatter_fin['receita_liquida'] / 1e6
    df_scatter_fin['resultado_mi'] = df_scatter_fin['resultado_liquido'] / 1e6
    
    fig_fin.add_trace(
        go.Scatter(
            x=df_scatter_fin['receita_mi'],
            y=df_scatter_fin['resultado_mi'],
            mode='markers',
            marker=dict(
                size=6,
                color=df_scatter_fin['margem_liquida_perc'],
                colorscale='RdYlGn',
                showscale=True,
                colorbar=dict(title="Margem %", x=1.15),
                opacity=0.6
            ),
            showlegend=False
        ),
        row=2, col=2
    )
    
    # Linha de break-even
    fig_fin.add_trace(
        go.Scatter(
            x=[0, df_scatter_fin['receita_mi'].max()],
            y=[0, 0],
            mode='lines',
            line=dict(color='gray', dash='dash'),
            showlegend=False
        ),
        row=2, col=2
    )
    
    # Atualizar eixos
    fig_fin.update_xaxes(title_text="Log10(Receita)", row=1, col=1)
    fig_fin.update_xaxes(title_text="Situação", row=1, col=2)
    fig_fin.update_xaxes(title_text="Receita (Mi R$)", row=2, col=2)
    
    fig_fin.update_yaxes(title_text="Frequência", row=1, col=1)
    fig_fin.update_yaxes(title_text="Qtd. Empresas", row=1, col=2)
    fig_fin.update_yaxes(title_text="ROE (%)", row=2, col=1)
    fig_fin.update_yaxes(title_text="Resultado (Mi R$)", row=2, col=2)
    
    fig_fin.update_layout(
        title='<b>Dashboard Financeiro - Análise de Receitas e Resultados</b>',
        height=900,
        showlegend=False
    )
    
    fig_fin.show()
    
    # ========================================================================
    # SUNBURST - HIERARQUIA DE DADOS
    # ========================================================================
    
    print("\n" + "="*80)
    print("🌞 VISUALIZAÇÃO HIERÁRQUICA (SUNBURST)")
    print("="*80)
    
    # Preparar dados para sunburst
    if 'cnae_secao_descricao' in df_dash.columns and 'classificacao_risco' in df_dash.columns:
        # Agregar por setor e risco
        df_sunburst = df_dash.groupby(['cnae_secao_descricao', 'classificacao_risco']).agg({
            'ativo_total': 'sum',
            'cnpj': 'count'
        }).reset_index()
        
        df_sunburst.columns = ['setor', 'risco', 'ativo', 'empresas']
        
        # Top 10 setores
        top_setores = df_dash['cnae_secao_descricao'].value_counts().head(10).index.tolist()
        df_sunburst = df_sunburst[df_sunburst['setor'].isin(top_setores)]
        
        # Criar hierarquia
        labels = ['Total']
        parents = ['']
        values = [df_sunburst['empresas'].sum()]
        
        for setor in df_sunburst['setor'].unique():
            setor_curto = setor[:30]
            labels.append(setor_curto)
            parents.append('Total')
            values.append(df_sunburst[df_sunburst['setor'] == setor]['empresas'].sum())
            
            for _, row in df_sunburst[df_sunburst['setor'] == setor].iterrows():
                labels.append(f"{setor_curto} - {row['risco']}")
                parents.append(setor_curto)
                values.append(row['empresas'])
        
        fig_sun = go.Figure(go.Sunburst(
            labels=labels,
            parents=parents,
            values=values,
            branchvalues="total",
            marker=dict(colorscale='RdYlGn_r')
        ))
        
        fig_sun.update_layout(
            title='<b>Hierarquia: Setores e Classificação de Risco</b>',
            height=700
        )
        
        fig_sun.show()

else:
    print("\n⚠️ Dados insuficientes para dashboard")

print("\n✅ Dashboard executivo concluído!")

In [ ]:
# ============================================================================
# CÉLULA 10: RELATÓRIO FINAL E RECOMENDAÇÕES
# ============================================================================

print("\n" + "="*80)
print("📋 RELATÓRIO FINAL - PROJETO ECD")
print("="*80)

# ============================================================================
# RESUMO EXECUTIVO
# ============================================================================

print("\n" + "="*80)
print("🎯 RESUMO EXECUTIVO")
print("="*80)

# Carregar métricas consolidadas
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW vw_resumo_executivo AS
SELECT 
    COUNT(DISTINCT ec.cnpj) AS total_empresas,
    COUNT(DISTINCT ec.id_ecd) AS total_ecds,
    CAST(MAX(ec.ano_referencia / 100) AS INT) AS ano_mais_recente,
    CAST(SUM(COALESCE(ind.ativo_total, 0)) AS DOUBLE) AS ativo_total_consolidado,
    CAST(SUM(COALESCE(ind.receita_liquida, 0)) AS DOUBLE) AS receita_total_consolidada,
    CAST(SUM(COALESCE(ind.resultado_liquido, 0)) AS DOUBLE) AS resultado_total_consolidado,
    CAST(AVG(COALESCE(ind.liquidez_corrente, 0)) AS DOUBLE) AS liquidez_media,
    CAST(AVG(COALESCE(ind.endividamento_geral, 0)) AS DOUBLE) AS endividamento_medio,
    CAST(AVG(COALESCE(ind.margem_liquida_perc, 0)) AS DOUBLE) AS margem_media,
    COUNT(DISTINCT CASE WHEN ec.empresa_grande_porte = 'Sim' THEN ec.cnpj END) AS empresas_grande_porte,
    COUNT(DISTINCT CASE WHEN sr.classificacao_risco IN ('RISCO CRÍTICO', 'RISCO ALTO') THEN ec.cnpj END) AS empresas_alto_risco,
    COUNT(DISTINCT CASE WHEN ind.resultado_liquido > 0 THEN ec.cnpj END) AS empresas_lucro,
    COUNT(DISTINCT CASE WHEN ind.resultado_liquido < 0 THEN ec.cnpj END) AS empresas_prejuizo
FROM teste.ecd_empresas_cadastro ec
LEFT JOIN teste.ecd_indicadores_financeiros ind
    ON ec.cnpj = ind.cnpj
    AND ec.ano_referencia = ind.ano_referencia
LEFT JOIN teste.ecd_score_risco_consolidado sr
    ON ec.cnpj = sr.cnpj
    AND ec.ano_referencia = sr.ano_referencia
""")

resumo = spark.sql("SELECT * FROM vw_resumo_executivo").toPandas()

if len(resumo) > 0:
    r = resumo.iloc[0]
    
    print(f"""
╔════════════════════════════════════════════════════════════════════════════╗
║                       RESUMO EXECUTIVO - ECD ONLINE                         ║
╠════════════════════════════════════════════════════════════════════════════╣
║                                                                            ║
║  📊 BASE DE DADOS                                                          ║
║     • Total de Empresas Analisadas: {r['total_empresas']:>32,}  ║
║     • Total de ECDs Processados: {r['total_ecds']:>35,}  ║
║     • Ano de Referência: {r['ano_mais_recente']:>43}  ║
║                                                                            ║
║  💰 VALORES CONSOLIDADOS                                                   ║
║     • Ativo Total: {f"R$ {r['ativo_total_consolidado']/1e9:>45,.2f} Bi"}  ║
║     • Receita Líquida Total: {f"R$ {r['receita_total_consolidada']/1e9:>36,.2f} Bi"}  ║
║     • Resultado Líquido Total: {f"R$ {r['resultado_total_consolidado']/1e9:>34,.2f} Bi"}  ║
║                                                                            ║
║  📊 INDICADORES MÉDIOS                                                     ║
║     • Liquidez Corrente Média: {r['liquidez_media']:>39,.2f}  ║
║     • Endividamento Médio: {r['endividamento_medio']:>43,.2%}  ║
║     • Margem Líquida Média: {r['margem_media']:>42,.2f}%  ║
║                                                                            ║
║  🏢 PERFIL DAS EMPRESAS                                                    ║
║     • Grande Porte: {r['empresas_grande_porte']:>50,} ({r['empresas_grande_porte']/r['total_empresas']*100:>4,.1f}%)  ║
║     • Com Lucro: {r['empresas_lucro']:>55,} ({r['empresas_lucro']/r['total_empresas']*100:>4,.1f}%)  ║
║     • Com Prejuízo: {r['empresas_prejuizo']:>52,} ({r['empresas_prejuizo']/r['total_empresas']*100:>4,.1f}%)  ║
║                                                                            ║
║  ⚠️  ANÁLISE DE RISCO                                                      ║
║     • Empresas de Alto Risco: {r['empresas_alto_risco']:>42,} ({r['empresas_alto_risco']/r['total_empresas']*100:>4,.1f}%)  ║
║                                                                            ║
╚════════════════════════════════════════════════════════════════════════════╝
    """)

# ============================================================================
# TOP EMPRESAS PARA FISCALIZAÇÃO
# ============================================================================

print("\n" + "="*80)
print("🎯 TOP 50 EMPRESAS PRIORITÁRIAS PARA FISCALIZAÇÃO")
print("="*80)

spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW vw_prioridades AS
SELECT 
    sr.cnpj,
    sr.razao_social,
    sr.uf,
    sr.de_cnae,
    sr.classificacao_risco,
    CAST(COALESCE(sr.score_risco_total, 0) AS DOUBLE) AS score_risco,
    CAST(COALESCE(sr.qtd_indicios_neaf, 0) AS DOUBLE) AS qtd_indicios,
    CAST(COALESCE(ind.ativo_total, 0) AS DOUBLE) AS ativo_total,
    CAST(COALESCE(ind.receita_liquida, 0) AS DOUBLE) AS receita_liquida,
    CAST(COALESCE(ind.liquidez_corrente, 0) AS DOUBLE) AS liquidez_corrente,
    CAST(COALESCE(ind.margem_liquida_perc, 0) AS DOUBLE) AS margem_liquida,
    CASE 
        WHEN sr.score_risco_total >= 7 AND ind.ativo_total >= 100000000 THEN 1
        WHEN sr.score_risco_total >= 7 OR ind.ativo_total >= 100000000 THEN 2
        WHEN sr.score_risco_total >= 5 THEN 3
        WHEN sr.score_risco_total >= 3 THEN 4
        ELSE 5
    END AS prioridade
FROM teste.ecd_score_risco_consolidado sr
LEFT JOIN teste.ecd_indicadores_financeiros ind
    ON sr.cnpj = ind.cnpj
    AND sr.ano_referencia = ind.ano_referencia
WHERE sr.score_risco_total >= 3
    OR sr.qtd_indicios_neaf >= 2
    OR ind.liquidez_corrente < 0.5
    OR (ind.resultado_liquido < 0 AND ind.ativo_total > 50000000)
ORDER BY prioridade ASC, score_risco DESC, ativo_total DESC
LIMIT 50
""")

total_prior = spark.sql("SELECT COUNT(*) as cnt FROM vw_prioridades").collect()[0]['cnt']

if total_prior > 0:
    df_prioridades = spark.sql("SELECT * FROM vw_prioridades").toPandas()
    
    print(f"\n✅ {len(df_prioridades)} empresas identificadas como prioritárias")
    
    # Estatísticas das prioridades
    print("\n📊 Distribuição por Prioridade:")
    dist_prior = df_prioridades['prioridade'].value_counts().sort_index()
    for prior, qtd in dist_prior.items():
        print(f"  • Prioridade {prior}: {qtd} empresas")
    
    # Criar tabela formatada
    df_export = df_prioridades.copy()
    df_export['ativo_mi'] = df_export['ativo_total'] / 1_000_000
    df_export['receita_mi'] = df_export['receita_liquida'] / 1_000_000
    
    print("\n📋 TOP 20 EMPRESAS PRIORITÁRIAS:")
    print(df_export.head(20)[[
        'cnpj', 'razao_social', 'uf', 'classificacao_risco', 
        'score_risco', 'qtd_indicios', 'ativo_mi', 'prioridade'
    ]].to_string(index=False))
    
    # Gráfico de prioridades
    fig = px.scatter(
        df_prioridades,
        x='score_risco',
        y='ativo_total',
        color='prioridade',
        size='qtd_indicios',
        hover_data=['razao_social', 'uf', 'de_cnae'],
        title='<b>Mapa de Priorização de Fiscalização</b>',
        labels={
            'score_risco': 'Score de Risco',
            'ativo_total': 'Ativo Total',
            'prioridade': 'Prioridade',
            'qtd_indicios': 'Indícios NEAF'
        },
        color_continuous_scale='RdYlGn_r',
        log_y=True
    )
    
    fig.update_layout(height=600)
    fig.show()

# ============================================================================
# PRINCIPAIS ACHADOS E INSIGHTS
# ============================================================================

print("\n" + "="*80)
print("🔍 PRINCIPAIS ACHADOS E INSIGHTS")
print("="*80)

# Análise de setores de risco
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW vw_setores_risco AS
SELECT 
    ec.cnae_secao_descricao,
    COUNT(DISTINCT ec.cnpj) AS qtd_empresas,
    CAST(AVG(COALESCE(sr.score_risco_total, 0)) AS DOUBLE) AS score_medio,
    COUNT(DISTINCT CASE WHEN sr.classificacao_risco IN ('RISCO CRÍTICO', 'RISCO ALTO') THEN ec.cnpj END) AS empresas_alto_risco,
    CAST(AVG(COALESCE(ind.margem_liquida_perc, 0)) AS DOUBLE) AS margem_media,
    CAST(AVG(COALESCE(ind.liquidez_corrente, 0)) AS DOUBLE) AS liquidez_media
FROM teste.ecd_empresas_cadastro ec
LEFT JOIN teste.ecd_score_risco_consolidado sr
    ON ec.cnpj = sr.cnpj
    AND ec.ano_referencia = sr.ano_referencia
LEFT JOIN teste.ecd_indicadores_financeiros ind
    ON ec.cnpj = ind.cnpj
    AND ec.ano_referencia = ind.ano_referencia
WHERE ec.cnae_secao_descricao IS NOT NULL
GROUP BY ec.cnae_secao_descricao
HAVING COUNT(DISTINCT ec.cnpj) >= 10
ORDER BY score_medio DESC
LIMIT 10
""")

df_setores_risco = spark.sql("SELECT * FROM vw_setores_risco").toPandas()

if len(df_setores_risco) > 0:
    print("\n🏭 TOP 10 SETORES COM MAIOR RISCO MÉDIO:")
    for idx, row in df_setores_risco.iterrows():
        perc_risco = (row['empresas_alto_risco'] / row['qtd_empresas']) * 100
        print(f"\n{idx+1}. {row['cnae_secao_descricao'][:60]}")
        print(f"   • Empresas: {row['qtd_empresas']:.0f}")
        print(f"   • Score Médio: {row['score_medio']:.2f}")
        print(f"   • Alto Risco: {row['empresas_alto_risco']:.0f} ({perc_risco:.1f}%)")
        print(f"   • Margem Média: {row['margem_media']:.2f}%")
        print(f"   • Liquidez Média: {row['liquidez_media']:.2f}")

# ============================================================================
# RECOMENDAÇÕES
# ============================================================================

print("\n" + "="*80)
print("💡 RECOMENDAÇÕES PARA AÇÃO FISCAL")
print("="*80)

print("""
╔════════════════════════════════════════════════════════════════════════════╗
║                            RECOMENDAÇÕES                                   ║
╠════════════════════════════════════════════════════════════════════════════╣
║                                                                            ║
║  1️⃣  PRIORIZAÇÃO DE FISCALIZAÇÃO                                          ║
║     → Focar nas empresas com Prioridade 1 e 2 (score ≥ 7 ou ativo alto)   ║
║     → Empresas com múltiplos indícios NEAF requerem atenção especial      ║
║     → Setores de alto risco devem ter fiscalização preventiva             ║
║                                                                            ║
║  2️⃣  MONITORAMENTO CONTÍNUO                                               ║
║     → Acompanhar empresas com liquidez < 1.0                              ║
║     → Monitorar empresas com endividamento > 70%                          ║
║     → Alertar sobre empresas em prejuízo recorrente                       ║
║                                                                            ║
║  3️⃣  ANÁLISE SETORIAL                                                     ║
║     → Investigar setores com score médio > 5.0                            ║
║     → Comparar empresas com benchmark setorial                            ║
║     → Identificar outliers por setor                                      ║
║                                                                            ║
║  4️⃣  VALIDAÇÃO DE INCONSISTÊNCIAS                                         ║
║     → Priorizar empresas com diferenças na equação contábil               ║
║     → Verificar empresas com variações atípicas ano a ano                 ║
║     → Cruzar dados ECD com outras obrigações acessórias                   ║
║                                                                            ║
║  5️⃣  INTELIGÊNCIA FISCAL                                                  ║
║     → Usar modelos de ML para predição de risco contínua                  ║
║     → Aplicar clustering para identificar padrões anômalos                ║
║     → Manter base de conhecimento atualizada                              ║
║                                                                            ║
╚════════════════════════════════════════════════════════════════════════════╝
""")

# ============================================================================
# CONCLUSÃO
# ============================================================================

print("\n" + "="*80)
print("✅ ANÁLISE CONCLUÍDA")
print("="*80)

print(f"""
╔════════════════════════════════════════════════════════════════════════════╗
║                            CONCLUSÃO DO PROJETO                            ║
╠════════════════════════════════════════════════════════════════════════════╣
║                                                                            ║
║  📊 Análise completa da base ECD realizada com sucesso                     ║
║  🤖 Modelos de Machine Learning treinados e validados                      ║
║  🔍 Clustering identificou grupos distintos de comportamento               ║
║  📈 Dashboards interativos gerados para análise visual                     ║
║  🎯 Lista de priorização para fiscalização preparada                       ║
║                                                                            ║
║  ✨ PRÓXIMOS PASSOS:                                                       ║
║     1. Revisar lista de empresas prioritárias                             ║
║     2. Aplicar critérios adicionais de seleção                            ║
║     3. Iniciar ações de fiscalização direcionada                          ║
║     4. Monitorar resultados e ajustar modelos                             ║
║     5. Integrar com outras bases de dados                                 ║
║                                                                            ║
║  📅 Data de execução: {datetime.now().strftime('%d/%m/%Y %H:%M:%S'):>44}  ║
║                                                                            ║
╚════════════════════════════════════════════════════════════════════════════╝

🎉 PROJETO ECD - ANÁLISE EXPLORATÓRIA E MACHINE LEARNING
   Auditor Fiscal da Receita Estadual de SC
   
   Desenvolvido com: Python, PySpark, Scikit-learn, XGBoost, Plotly
   
   Para dúvidas ou sugestões, consulte a equipe de Data Science.
""")

print("\n" + "="*80)
print("🙏 OBRIGADO POR UTILIZAR ESTE PROJETO!")
print("="*80)